<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Quant_Insights_updated_19thjune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
data_=pd.read_csv('/content/gdrive/MyDrive/Final_Merged_Data/all_data_combined.csv')
data_

,datetime,atm,open,high,low,close,volume,oi,max_pain_strike,intraday_pcr,...,highest_above_vah,lowest_below_val,gap_type,gap_filled,3_day_vah,5_day_vah,3_day_val,5_day_val,3_day_poc,5_day_poc
0,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 09:25:00,25570.00,25665.30,25665.30,25655.10,25655.10,240.0,0.0,25700,0.7420,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 09:30:00,25570.00,25647.70,25647.70,25647.70,25647.70,240.0,0.0,25600,0.7683,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 09:35:00,25570.00,25647.70,25647.70,25647.00,25647.00,440.0,0.0,25700,0.7973,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92694,2022-12-30 15:10:00,43501.05,43320.05,43349.70,43302.45,43322.00,2650.0,168225.0,42500,0.6826,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92695,2022-12-30 15:15:00,43501.05,43317.30,43331.95,43290.00,43310.00,4950.0,168625.0,42000,0.6204,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92696,2022-12-30 15:20:00,43501.05,43307.35,43443.95,43293.25,43432.90,6600.0,168750.0,42500,0.5152,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92697,2022-12-30 15:25:00,43501.05,43433.60,43443.70,43377.05,43403.40,6100.0,168500.0,42000,0.4360,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7


# Mean Reversion from Outside VAH - VAL

all thresholds VAH

In [ ]:
import pandas as pd
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)

def calculate_reversions(data, threshold_percentage):
    move_out_count = 0
    reversion_count = 0
    currently_outside = False

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']

        if prev_vah is not None:
            threshold = prev_vah * threshold_percentage
            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False

    return move_out_count, reversion_count

thresholds = [0.0005,0.00075, 0.001, 0.0015]
results = {}
for threshold in thresholds:
    move_out_count, reversion_count = calculate_reversions(data, threshold)
    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
Threshold 0.075%:
  Total move outs: 372
  Total reversions: 90
Threshold 0.1%:
  Total move outs: 318
  Total reversions: 49
Threshold 0.15%:
  Total move outs: 269
  Total reversions: 20
Threshold 0.2%:
  Total move outs: 227
  Total reversions: 6


In [39]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
import pandas as pd
'''
# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

def calculate_short_selling_profit_loss(data):
    short_selling_results = []

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_local_max = data.loc[i, 'prev_local_max']

        if prev_local_max is not None:
            entry_price = prev_local_max

            if close_price <= entry_price:
                exit_price = close_price
                percentage_change = ((entry_price - exit_price) / entry_price) * 100
                short_selling_results.append(percentage_change)

    return short_selling_results

short_selling_results = calculate_short_selling_profit_loss(data)

daily_short_selling_results = {}
total_short_selling_profit_loss = 0

for i, percentage_change in enumerate(short_selling_results):
    day = data.loc[i+1, 'datetime'].date()
    if day not in daily_short_selling_results:
        daily_short_selling_results[day] = []
    daily_short_selling_results[day].append(percentage_change)
    total_short_selling_profit_loss += percentage_change

print("Daily Short Selling Profit/Loss (%):")
for day, results in daily_short_selling_results.items():
    daily_profit_loss = sum(results)
    print(f"{day}: {daily_profit_loss:.2f}%")

print(f"\nTotal Short Selling Profit/Loss (%): {total_short_selling_profit_loss:.2f}%")
'''
def calculate_reversions(data, threshold_percentage):
    move_out_count = 0
    reversion_count = 0
    local_max_reach_count = 0
    reversion_after_local_max = 0
    currently_outside = False
    reached_local_max = False
    reversion_gap = []

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']
        prev_local_max = data.loc[i, 'prev_local_max']

        if prev_vah is not None and prev_local_max is not None:
            threshold = prev_vah * threshold_percentage

            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
                    reached_local_max = False
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False
                    if reached_local_max:
                        reversion_after_local_max += 1
                        gap = (prev_local_max - prev_vah) / prev_vah
                        reversion_gap.append(gap)
                        reached_local_max = False

            if close_price >= prev_local_max and currently_outside and not reached_local_max:
                local_max_reach_count += 1
                reached_local_max = True

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False
                reached_local_max = False

    average_reversion_gap = (sum(reversion_gap) / len(reversion_gap)) * 100 if reversion_gap else None

    return {
        'move_out_count': move_out_count,
        'reversion_count': reversion_count,
        'local_max_reach_count': local_max_reach_count,
        'reversion_after_local_max': reversion_after_local_max,
        'average_reversion_gap': average_reversion_gap,
        'reversion_gaps': reversion_gap
    }

thresholds = [0.0005, 0.00075, 0.001, 0.0015]
results = {}
for threshold in thresholds:
    stats = calculate_reversions(data, threshold)
    move_out_count = stats['move_out_count']
    reversion_count = stats['reversion_count']
    local_max_reach_count = stats['local_max_reach_count']
    reversion_after_local_max = stats['reversion_after_local_max']
    average_reversion_gap = stats['average_reversion_gap']

    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count,
        'Reached local max': local_max_reach_count,
        'Reverted after reaching local max': reversion_after_local_max,
        'Average reversion gap (%)': average_reversion_gap,
        'Daily reversion gaps (%)': [gap * 100 for gap in stats['reversion_gaps']]
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")
    print(f"  Reached local max: {result['Reached local max']}")
    print(f"  Reverted after reaching local max: {result['Reverted after reaching local max']}")


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
  Reached local max: 439
  Reverted after reaching local max: 144
Threshold 0.075%:
  Total move outs: 372
  Total reversions: 90
  Reached local max: 361
  Reverted after reaching local max: 79
Threshold 0.1%:
  Total move outs: 318
  Total reversions: 49
  Reached local max: 315
  Reverted after reaching local max: 46
Threshold 0.15%:
  Total move outs: 269
  Total reversions: 20
  Reached local max: 269
  Reverted after reaching local max: 20


In [12]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price):
        self.entry_price = entry_price
        self.exit_price = exit_price

def mean_reversion_strategy(data):
    trades = []
    flag = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if candle['close'] > prev_candle['prev_vah'] and candle['close'] >= prev_candle['prev_local_max']:
            if not flag:
                entry_price = candle['close']  # Entry price for short sell
                flag = True

        elif flag and candle['datetime'].date() != prev_candle['datetime'].date():
            exit_price = prev_candle['close']  # Exit price at the end of the day
            profit_percentage = ((entry_price - exit_price) / entry_price) * 100  # Corrected calculation
            daily_profit_percentage.append({'Date': prev_candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price))
            flag = False

    total_profit_percentage = sum([trade.entry_price - trade.exit_price for trade in trades]) / trades[0].entry_price * 100  # Total profit percentage calculation

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = mean_reversion_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)


Total profit percentage: 55.39659224441809
Total trades taken: 462
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-03                 0.352135
1    2018-01-08                -0.743055
2    2018-01-09                 0.019395
3    2018-01-15                -1.262524
4    2018-01-18                 0.498407
..          ...                      ...
457  2022-12-09                 0.251179
458  2022-12-14                -0.341252
459  2022-12-21                 1.535160
460  2022-12-27                -0.178945
461  2022-12-28                 0.485996

[462 rows x 2 columns]
< -2%: 45
-2% to 0%: 101
0% to 1%: 212
1% to 2%: 73
> 2%: 31


In [17]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, entry_date, exit_date, trade_type):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.entry_date = entry_date
        self.exit_date = exit_date
        self.trade_type = trade_type

def mean_reversion_strategy(data):
    trades = []
    flag = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    # Group data by date
    data['date'] = data['datetime'].dt.date
    grouped = data.groupby('date')

    for date, group in grouped:
        prev_date = (pd.Timestamp(date) - pd.Timedelta(days=1)).date()
        prev_day_data = data[data['datetime'].dt.date == prev_date]

        if not prev_day_data.empty:
            prev_candle = prev_day_data.iloc[-1]

            for _, candle in group.iterrows():
                # Short Sell Strategy
                if candle['close'] > prev_candle['prev_vah'] and candle['close'] >= prev_candle['prev_local_max']:
                    if not flag:
                        entry_price = candle['close']
                        entry_date = candle['datetime']
                        flag = True

                elif flag and candle['datetime'].date() != prev_candle['datetime'].date():
                    exit_price = prev_candle['close']
                    exit_date = prev_candle['datetime']
                    profit_percentage = ((entry_price - exit_price) / entry_price) * 100
                    daily_profit_percentage.append({'Date': date, 'Daily Profit Percentage': profit_percentage})
                    trades.append(Trade(entry_price, exit_price, entry_date, exit_date, "short"))
                    flag = False

    if trades:
        total_profit_percentage = sum(
            [(trade.exit_price - trade.entry_price) if trade.trade_type == "short" else (trade.entry_price - trade.exit_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = mean_reversion_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)
# Group trades and profits by year
profit_distribution = {}
for trade in trades:
    year = trade.entry_date.year
    if year not in profit_distribution:
        profit_distribution[year] = []
    if trade.trade_type == "short":
        profit_distribution[year].append(trade.entry_price - trade.exit_price)
    else:
        profit_distribution[year].append(trade.exit_price - trade.entry_price)

# Calculate total profits per year
yearly_total_profits = {year: sum(profits) for year, profits in profit_distribution.items()}

print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_total_profits.items():
    print(f"{year}: {abs(profit)}")

# Calculate total profits in percentage per year
initial_capital = trades[0].entry_price  # Assuming the initial capital is the first entry price
yearly_total_profits_percentage = {year: (profit / initial_capital) * 100 for year, profit in yearly_total_profits.items()}

print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_total_profits_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")


Total profit percentage: -244.13885624755204
Total trades taken: 1039
Daily profit percentage:
             Date  Daily Profit Percentage
0     2018-01-03                 0.339600
1     2018-01-03                 0.359115
2     2018-01-03                 0.319688
3     2018-01-04                 0.434424
4     2018-01-09                -0.822309
...          ...                      ...
1034  2022-12-30                 0.049586
1035  2022-12-30                 0.062693
1036  2022-12-30                 0.145626
1037  2022-12-30                 0.095331
1038  2022-12-30                 0.151479

[1039 rows x 2 columns]
< -2%: 42
-2% to 0%: 165
0% to 1%: 715
1% to 2%: 92
> 2%: 25

Yearly total profits (Absolute Value):
2018: 3601.899999999976
2019: 13748.399999999987
2020: 5649.4999999999745
2021: 38510.35000000008
2022: 818.5000000000364

Yearly total profits (Percentage):
2018: 14.11%
2019: 53.85%
2020: 22.13%
2021: 150.84%
2022: 3.21%


In [4]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price):
        self.entry_price = entry_price
        self.exit_price = exit_price

def mean_reversion_strategy(data):
    trades = []
    flag = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if candle['close'] < prev_candle['prev_val'] and candle['low'] <= prev_candle['prev_local_min']:
            if not flag:
                entry_price = candle['low']  # Entry price for long position
                flag = True

        elif flag and candle['datetime'].date() != prev_candle['datetime'].date():
            exit_price = prev_candle['high']  # Exit price at the end of the day
            profit_percentage = ((exit_price - entry_price) / entry_price) * 100
            daily_profit_percentage.append({'Date': prev_candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price))
            flag = False

    total_profit_percentage = sum([trade.exit_price - trade.entry_price for trade in trades]) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = mean_reversion_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)
# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)


Total profit percentage: 142.95775199874555
Total trades taken: 714
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-01                -0.095234
1    2018-01-02                -0.071454
2    2018-01-03                -0.075789
3    2018-01-04                 0.522068
4    2018-01-05                 0.390320
..          ...                      ...
709  2022-12-20                 0.436782
710  2022-12-21                 0.775460
711  2022-12-26                 0.536839
712  2022-12-27                 1.033771
713  2022-12-29                 1.016208

[714 rows x 2 columns]
< -2%: 48
-2% to 0%: 184
0% to 1%: 316
1% to 2%: 116
> 2%: 50


In [20]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    flag = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not flag:
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] and candle['close'] >= prev_candle['prev_local_max']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                flag = True

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] and candle['low'] <= prev_candle['prev_local_min']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                flag = True

        elif flag and candle['datetime'].date() != prev_candle['datetime'].date():
            if trade_type == "short":
                exit_price = prev_candle['close']
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
            elif trade_type == "long":
                exit_price = prev_candle['close']
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100

            daily_profit_percentage.append({'Date': prev_candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price, trade_type, entry_date, prev_candle['datetime']))
            flag = False

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")



Total profit percentage: 382.22805642633296
Total trades taken: 1126
Daily profit percentage:
             Date  Daily Profit Percentage
0     2018-01-01                -0.146552
1     2018-01-02                -0.108751
2     2018-01-03                 0.371257
3     2018-01-04                 0.494018
4     2018-01-05                -0.301600
...          ...                      ...
1121  2022-12-23                -0.907642
1122  2022-12-26                 1.153941
1123  2022-12-27                -0.063835
1124  2022-12-28                -0.105146
1125  2022-12-29                 1.562895

[1126 rows x 2 columns]
< -2%: 37
-2% to 0%: 371
0% to 1%: 478
1% to 2%: 171
> 2%: 69

Yearly total profits (Percentage):
2018: 45.32%
2019: 73.96%
2020: 105.56%
2021: 52.46%
2022: 61.12%

Yearly total profits (Absolute Value):
2018: 11566.31
2019: 18875.45
2020: 26940.08
2021: 13388.26
2022: 15598.98


In [21]:
import pandas as pd
#################################################################################################################################################################
# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    flag = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not flag:
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] and candle['close'] >= prev_candle['prev_local_max']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                flag = True

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] and candle['low'] <= prev_candle['prev_local_min']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                flag = True

        elif flag and candle['datetime'].date() != prev_candle['datetime'].date():
            if trade_type == "short":
                exit_price = prev_candle['close']
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
            elif trade_type == "long":
                exit_price = prev_candle['close']
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100

            daily_profit_percentage.append({'Date': prev_candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price, trade_type, entry_date, prev_candle['datetime']))
            flag = False

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")

# Group daily profits by year and month
daily_profit['Month'] = daily_profit['Date'].dt.month
monthly_profit_percentage = daily_profit.groupby(['Year', 'Month'])['Daily Profit Percentage'].sum()

# Print the monthly profit percentages
print("\nMonthly total profits (Percentage):")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")


Total profit percentage: 382.22805642633296
Total trades taken: 1126
Daily profit percentage:
             Date  Daily Profit Percentage
0     2018-01-01                -0.146552
1     2018-01-02                -0.108751
2     2018-01-03                 0.371257
3     2018-01-04                 0.494018
4     2018-01-05                -0.301600
...          ...                      ...
1121  2022-12-23                -0.907642
1122  2022-12-26                 1.153941
1123  2022-12-27                -0.063835
1124  2022-12-28                -0.105146
1125  2022-12-29                 1.562895

[1126 rows x 2 columns]

Yearly total profits (Percentage):
2018: 45.32%
2019: 73.96%
2020: 105.56%
2021: 52.46%
2022: 61.12%

Yearly total profits (Absolute Value):
2018: 11566.31
2019: 18875.45
2020: 26940.08
2021: 13388.26
2022: 15598.98

Monthly total profits (Percentage):
2018-01: 3.28%
2018-02: 5.66%
2018-03: 5.96%
2018-04: 5.85%
2018-05: 3.42%
2018-06: 2.92%
2018-07: 0.53%
2018-08: 0.60%
20

In [23]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    in_trade = False
    total_profit_percentage = 0
    daily_profit_percentage = []

    for i in range(1, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        if not in_trade:
            # Short Sell Strategy
            if candle['close'] > prev_candle['prev_vah'] and candle['close'] >= prev_candle['prev_local_max']:
                entry_price = candle['close']
                trade_type = "short"
                entry_date = candle['datetime']
                in_trade = True

            # Long Buy Strategy
            elif candle['close'] < prev_candle['prev_val'] and candle['low'] <= prev_candle['prev_local_min']:
                entry_price = candle['close']
                trade_type = "long"
                entry_date = candle['datetime']
                in_trade = True

        if in_trade and (candle['datetime'].date() != prev_candle['datetime'].date() or i == len(data) - 1):
            if trade_type == "short":
                exit_price = prev_candle['close']
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
            elif trade_type == "long":
                exit_price = prev_candle['close']
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100

            daily_profit_percentage.append({'Date': prev_candle['datetime'].date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price, trade_type, entry_date, prev_candle['datetime']))
            in_trade = False

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Ensure 'Date' column is datetime type
daily_profit['Date'] = pd.to_datetime(daily_profit['Date'])

# Group daily profits by year
daily_profit['Year'] = daily_profit['Date'].dt.year
yearly_profit_percentage = daily_profit.groupby('Year')['Daily Profit Percentage'].sum()

# Print the yearly profit percentages
print("\nYearly total profits (Percentage):")
for year, profit_percentage in yearly_profit_percentage.items():
    print(f"{year}: {profit_percentage:.2f}%")

# Calculate absolute yearly profits based on entry price
yearly_absolute_profits = {}
initial_entry_price = trades[0].entry_price
for year, group in daily_profit.groupby('Year'):
    absolute_profit = initial_entry_price * group['Daily Profit Percentage'].sum() / 100
    yearly_absolute_profits[year] = abs(absolute_profit)

# Print the yearly absolute profits
print("\nYearly total profits (Absolute Value):")
for year, profit in yearly_absolute_profits.items():
    print(f"{year}: {profit:.2f}")

# Group daily profits by year and month
daily_profit['Month'] = daily_profit['Date'].dt.month
monthly_profit_percentage = daily_profit.groupby(['Year', 'Month'])['Daily Profit Percentage'].sum()

# Print the monthly profit percentages
print("\nMonthly total profits (Percentage):")
for (year, month), profit_percentage in monthly_profit_percentage.items():
    print(f"{year}-{month:02d}: {profit_percentage:.2f}%")


Total profit percentage: 383.85384012539254
Total trades taken: 1128
Daily profit percentage:
             Date  Daily Profit Percentage
0     2018-01-01                -0.146552
1     2018-01-02                -0.108751
2     2018-01-03                 0.371257
3     2018-01-04                 0.494018
4     2018-01-05                -0.301600
...          ...                      ...
1123  2022-12-26                 1.153941
1124  2022-12-27                -0.063835
1125  2022-12-28                -0.105146
1126  2022-12-29                 1.562895
1127  2022-12-30                 0.521191

[1128 rows x 2 columns]

Yearly total profits (Percentage):
2018: 45.32%
2019: 74.57%
2020: 105.56%
2021: 52.46%
2022: 61.65%

Yearly total profits (Absolute Value):
2018: 11566.31
2019: 19031.34
2020: 26940.08
2021: 13388.26
2022: 15731.99

Monthly total profits (Percentage):
2018-01: 3.28%
2018-02: 5.66%
2018-03: 5.96%
2018-04: 5.85%
2018-05: 3.42%
2018-06: 2.92%
2018-07: 0.53%
2018-08: 0.60%
20

In [10]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    flag = False
    trade_type = None
    entry_price = None
    total_profit_percentage = 0
    daily_profit_percentage = []

    grouped = data.groupby(data['datetime'].dt.date)

    for date, group in grouped:
        first_candle = group.iloc[0]

        if not flag:
            prev_date = (pd.Timestamp(date) - pd.Timedelta(days=1)).date()
            prev_day_data = data[data['datetime'].dt.date == prev_date]

            if not prev_day_data.empty:
                prev_candle = prev_day_data.iloc[-1]

                # Short Sell Strategy
                if first_candle['close'] > prev_candle['prev_vah'] and first_candle['high'] >= prev_candle['prev_local_max']:
                    entry_price = first_candle['high']
                    trade_type = "short"
                    entry_date = first_candle['datetime']
                    flag = True

                # Long Buy Strategy
                elif first_candle['close'] < prev_candle['prev_val'] and first_candle['low'] <= prev_candle['prev_local_min']:
                    entry_price = first_candle['low']
                    trade_type = "long"
                    entry_date = first_candle['datetime']
                    flag = True

        if flag and (group['datetime'].dt.date.iloc[-1] != entry_date.date() or group.index[-1] == data.index[-1]):
            last_candle = group.iloc[-1]
            if trade_type == "short":
                exit_price = last_candle['low']
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
            elif trade_type == "long":
                exit_price = last_candle['high']
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100

            daily_profit_percentage.append({'Date': entry_date.date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price, trade_type, entry_date, last_candle['datetime']))
            flag = False

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)


Total profit percentage: 198.03329860648714
Total trades taken: 370
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-03                -0.051307
1    2018-01-05                -0.538992
2    2018-01-09                -0.224461
3    2018-01-11                 0.719312
4    2018-01-17                 2.513261
..          ...                      ...
365  2022-12-15                -1.697440
366  2022-12-20                -1.253518
367  2022-12-23                 1.295082
368  2022-12-27                 0.135089
369  2022-12-29                 1.442688

[370 rows x 2 columns]
< -2%: 33
-2% to 0%: 118
0% to 1%: 93
1% to 2%: 57
> 2%: 69


In [11]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)
data['prev_local_min'] = data['low'].shift(1)

class Trade:
    def __init__(self, entry_price, exit_price, trade_type, entry_date, exit_date):
        self.entry_price = entry_price
        self.exit_price = exit_price
        self.trade_type = trade_type
        self.entry_date = entry_date
        self.exit_date = exit_date

def combined_strategy(data):
    trades = []
    flag = False
    trade_type = None
    entry_price = None
    total_profit_percentage = 0
    daily_profit_percentage = []

    grouped = data.groupby(data['datetime'].dt.date)

    for date, group in grouped:
        first_candle = group.iloc[0]

        if not flag:
            prev_date = (pd.Timestamp(date) - pd.Timedelta(days=1)).date()
            prev_day_data = data[data['datetime'].dt.date == prev_date]

            if not prev_day_data.empty:
                prev_candle = prev_day_data.iloc[-1]

                # Short Sell Strategy
                if first_candle['close'] > prev_candle['prev_vah'] and first_candle['close'] >= prev_candle['prev_local_max']:
                    entry_price = first_candle['close']
                    trade_type = "short"
                    entry_date = first_candle['datetime']
                    flag = True

                # Long Buy Strategy
                elif first_candle['close'] < prev_candle['prev_val'] and first_candle['close'] <= prev_candle['prev_local_min']:
                    entry_price = first_candle['close']
                    trade_type = "long"
                    entry_date = first_candle['datetime']
                    flag = True

        if flag and (group['datetime'].dt.date.iloc[-1] != entry_date.date() or group.index[-1] == data.index[-1]):
            last_candle = group.iloc[-1]
            if trade_type == "short":
                exit_price = last_candle['close']
                profit_percentage = ((entry_price - exit_price) / entry_price) * 100
            elif trade_type == "long":
                exit_price = last_candle['close']
                profit_percentage = ((exit_price - entry_price) / entry_price) * 100

            daily_profit_percentage.append({'Date': entry_date.date(), 'Daily Profit Percentage': profit_percentage})
            trades.append(Trade(entry_price, exit_price, trade_type, entry_date, last_candle['datetime']))
            flag = False

    if trades:
        total_profit_percentage = sum(
            [(trade.entry_price - trade.exit_price) if trade.trade_type == "short" else (trade.exit_price - trade.entry_price) for trade in trades]
        ) / trades[0].entry_price * 100

    return trades, total_profit_percentage, pd.DataFrame(daily_profit_percentage)

trades, total_profit, daily_profit = combined_strategy(data)

print("Total profit percentage:", total_profit)
print("Total trades taken:", len(trades))
print("Daily profit percentage:\n", daily_profit)

# Extracting daily profit percentages from the DataFrame
daily_profits = daily_profit['Daily Profit Percentage']

# Define the ranges
ranges = {
    "< -2%": (-float('inf'), -2),
    "-2% to 0%": (-2, 0),
    "0% to 1%": (0, 1),
    "1% to 2%": (1, 2),
    "> 2%": (2, float('inf'))
}

# Initialize a dictionary to store the count of profits in each range
profit_count = {range_name: 0 for range_name in ranges}

# Count the occurrences of profits in each range
for profit in daily_profits:
    for range_name, (lower, upper) in ranges.items():
        if lower < profit <= upper:
            profit_count[range_name] += 1

# Print the count of profits in each range
for range_name, count in profit_count.items():
    print(range_name + ":", count)


Total profit percentage: 69.28339208773974
Total trades taken: 348
Daily profit percentage:
            Date  Daily Profit Percentage
0    2018-01-03                -0.157070
1    2018-01-05                -0.609670
2    2018-01-09                -0.254287
3    2018-01-11                 0.550044
4    2018-01-17                 2.134862
..          ...                      ...
343  2022-12-15                -1.837034
344  2022-12-20                -1.309364
345  2022-12-23                 1.119569
346  2022-12-27                 0.092958
347  2022-12-29                 1.328318

[348 rows x 2 columns]
< -2%: 40
-2% to 0%: 130
0% to 1%: 79
1% to 2%: 45
> 2%: 54


target and sl

In [13]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

def calculate_reversions_and_profit(data, threshold_percentage, stop_loss_percentage=0.015):
    move_out_count = 0
    reversion_count = 0
    local_max_reach_count = 0
    reversion_after_local_max = 0
    currently_outside = False
    reached_local_max = False
    reversion_gap = []

    total_profit = 0
    total_target_hit_count = 0
    total_stop_loss_count = 0

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']
        prev_val = data.loc[i, 'prev_val']
        prev_local_max = data.loc[i, 'prev_local_max']

        if prev_vah is not None and prev_val is not None and prev_local_max is not None:
            threshold = prev_vah * threshold_percentage
            target_price = (prev_vah + prev_val) / 2
            stop_loss_price = prev_local_max * (1 + stop_loss_percentage)

            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
                    reached_local_max = False
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False
                    if reached_local_max:
                        reversion_after_local_max += 1
                        gap = (prev_local_max - prev_vah) / prev_vah
                        reversion_gap.append(gap)
                        reached_local_max = False

            if close_price >= prev_local_max and currently_outside and not reached_local_max:
                local_max_reach_count += 1
                reached_local_max = True

                # Simulate the short position
                position_opened = True
                for j in range(i+1, len(data)):
                    future_close_price = data.loc[j, 'close']
                    if future_close_price <= target_price:
                        profit = prev_local_max - target_price
                        total_profit += profit
                        total_target_hit_count += 1
                        position_opened = False
                        break
                    elif future_close_price >= stop_loss_price:
                        loss = stop_loss_price - prev_local_max
                        total_profit -= loss
                        total_stop_loss_count += 1
                        position_opened = False
                        break

                if position_opened:
                    future_close_price = data.loc[j, 'close']
                    profit_or_loss = prev_local_max - future_close_price
                    total_profit += profit_or_loss

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False
                reached_local_max = False

    average_reversion_gap = (sum(reversion_gap) / len(reversion_gap)) * 100 if reversion_gap else None

    return {
        'move_out_count': move_out_count,
        'reversion_count': reversion_count,
        'local_max_reach_count': local_max_reach_count,
        'reversion_after_local_max': reversion_after_local_max,
        'average_reversion_gap': average_reversion_gap,
        'reversion_gaps': reversion_gap,
        'total_profit': total_profit,
        'total_target_hit_count': total_target_hit_count,
        'total_stop_loss_count': total_stop_loss_count
    }

thresholds = [0.0005, 0.00075, 0.001, 0.0015]
results = {}
for threshold in thresholds:
    stats = calculate_reversions_and_profit(data, threshold)
    move_out_count = stats['move_out_count']
    reversion_count = stats['reversion_count']
    local_max_reach_count = stats['local_max_reach_count']
    reversion_after_local_max = stats['reversion_after_local_max']
    average_reversion_gap = stats['average_reversion_gap']
    total_profit = stats['total_profit']
    total_target_hit_count = stats['total_target_hit_count']
    total_stop_loss_count = stats['total_stop_loss_count']

    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count,
        'Reached local max': local_max_reach_count,
        'Reverted after reaching local max': reversion_after_local_max,
        'Average reversion gap (%)': average_reversion_gap,
        'Daily reversion gaps (%)': [gap * 100 for gap in stats['reversion_gaps']],
        'Total Profit': total_profit,
        'Total Target Hit Count': total_target_hit_count,
        'Total Stop Loss Count': total_stop_loss_count
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")
    print(f"  Reached local max: {result['Reached local max']}")
    print(f"  Reverted after reaching local max: {result['Reverted after reaching local max']}")
    print(f"  Average reversion gap: {result['Average reversion gap (%)']:.2f}%")
    print(f"  Daily reversion gaps: {result['Daily reversion gaps (%)']}")
    print(f"  Total Profit: {result['Total Profit']}")
    print(f"  Total Target Hit Count: {result['Total Target Hit Count']}")
    print(f"  Total Stop Loss Count: {result['Total Stop Loss Count']}")


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
  Reached local max: 439
  Reverted after reaching local max: 144
  Average reversion gap: 0.15%
  Daily reversion gaps: [0.07785937141161536, 0.09065129468643104, 0.10943426042983853, 0.05062970698058236, 0.27723169657622027, 0.0965677653384358, 0.1306897514660655, 0.1450741921145407, 0.05839236039330404, 0.16816439094607322, 0.12116000938012976, 0.059070199961391165, 0.059070199961391165, 0.0637216431209133, 0.0637216431209133, 0.41579036191426244, 0.41579036191426244, 0.13657629384244618, 0.13657629384244618, 0.09213353355665667, 0.0878508325738441, 0.19851672928321162, 0.10657454705817501, 0.11328574097795327, 0.0892026673005526, 0.15594270675135444, 0.05541530693923037, 0.05481947380179316, 0.19696245098892165, 0.09207094735732045, 0.15954659814869532, 0.06145433890903323, 0.21692087262775184, 0.10711847762830007, 0.056580668183153285, 0.16068903458028022, 0.1734724297374788, 0.10252654057406789, 0.08341402052897087, 

In [19]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

def calculate_reversions_and_profit(data, threshold_percentage, stop_loss_percentage=0.03):
    move_out_count = 0
    reversion_count = 0
    local_max_reach_count = 0
    reversion_after_local_max = 0
    currently_outside = False
    reached_local_max = False
    reversion_gap = []

    total_profit = 0
    total_target_hit_count = 0
    total_stop_loss_count = 0
    total_initial_position = 0  # To track the total initial position amount for percentage calculation

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']
        prev_val = data.loc[i, 'prev_val']
        prev_local_max = data.loc[i, 'prev_local_max']

        if prev_vah is not None and prev_val is not None and prev_local_max is not None:
            threshold = prev_vah * threshold_percentage
            #target_price = (prev_vah + prev_val) / 2
            target_price = prev_val
            stop_loss_price = prev_local_max * (1 + stop_loss_percentage)

            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
                    reached_local_max = False
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False
                    if reached_local_max:
                        reversion_after_local_max += 1
                        gap = (prev_local_max - prev_vah) / prev_vah
                        reversion_gap.append(gap)
                        reached_local_max = False

            if close_price >= prev_local_max and currently_outside and not reached_local_max:
                local_max_reach_count += 1
                reached_local_max = True

                # Simulate the short position
                initial_position = prev_local_max
                total_initial_position += initial_position
                position_opened = True
                for j in range(i+1, len(data)):
                    future_close_price = data.loc[j, 'close']
                    if future_close_price <= target_price:
                        profit = prev_local_max - target_price
                        total_profit += profit
                        total_target_hit_count += 1
                        position_opened = False
                        break
                    elif future_close_price >= stop_loss_price:
                        loss = stop_loss_price - prev_local_max
                        total_profit -= loss
                        total_stop_loss_count += 1
                        position_opened = False
                        break

                if position_opened:
                    future_close_price = data.loc[j, 'close']
                    profit_or_loss = prev_local_max - future_close_price
                    total_profit += profit_or_loss

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False
                reached_local_max = False

    average_reversion_gap = (sum(reversion_gap) / len(reversion_gap)) * 100 if reversion_gap else None
    total_profit_percentage = (total_profit / total_initial_position) * 100 if total_initial_position != 0 else None

    return {
        'move_out_count': move_out_count,
        'reversion_count': reversion_count,
        'local_max_reach_count': local_max_reach_count,
        'reversion_after_local_max': reversion_after_local_max,
        'average_reversion_gap': average_reversion_gap,
        'reversion_gaps': reversion_gap,
        'total_profit': total_profit_percentage,
        'total_target_hit_count': total_target_hit_count,
        'total_stop_loss_count': total_stop_loss_count
    }

thresholds = [0.0005, 0.00075, 0.001, 0.0015]
results = {}
for threshold in thresholds:
    stats = calculate_reversions_and_profit(data, threshold)
    move_out_count = stats['move_out_count']
    reversion_count = stats['reversion_count']
    local_max_reach_count = stats['local_max_reach_count']
    reversion_after_local_max = stats['reversion_after_local_max']
    average_reversion_gap = stats['average_reversion_gap']
    total_profit_percentage = stats['total_profit']
    total_target_hit_count = stats['total_target_hit_count']
    total_stop_loss_count = stats['total_stop_loss_count']

    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count,
        'Reached local max': local_max_reach_count,
        'Reverted after reaching local max': reversion_after_local_max,
        'Average reversion gap (%)': average_reversion_gap,
        'Daily reversion gaps (%)': [gap * 100 for gap in stats['reversion_gaps']],
        'Total Profit (%)': total_profit_percentage,
        'Total Target Hit Count': total_target_hit_count,
        'Total Stop Loss Count': total_stop_loss_count
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")
    print(f"  Reached local max: {result['Reached local max']}")
    print(f"  Reverted after reaching local max: {result['Reverted after reaching local max']}")
    print(f"  Average reversion gap: {result['Average reversion gap (%)']:.2f}%")
    print(f"  Daily reversion gaps: {result['Daily reversion gaps (%)']}")
    print(f"  Total Profit: {result['Total Profit (%)']:.2f}%")
    print(f"  Total Target Hit Count: {result['Total Target Hit Count']}")
    print(f"  Total Stop Loss Count: {result['Total Stop Loss Count']}")


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
  Reached local max: 439
  Reverted after reaching local max: 144
  Average reversion gap: 0.15%
  Daily reversion gaps: [0.07785937141161536, 0.09065129468643104, 0.10943426042983853, 0.05062970698058236, 0.27723169657622027, 0.0965677653384358, 0.1306897514660655, 0.1450741921145407, 0.05839236039330404, 0.16816439094607322, 0.12116000938012976, 0.059070199961391165, 0.059070199961391165, 0.0637216431209133, 0.0637216431209133, 0.41579036191426244, 0.41579036191426244, 0.13657629384244618, 0.13657629384244618, 0.09213353355665667, 0.0878508325738441, 0.19851672928321162, 0.10657454705817501, 0.11328574097795327, 0.0892026673005526, 0.15594270675135444, 0.05541530693923037, 0.05481947380179316, 0.19696245098892165, 0.09207094735732045, 0.15954659814869532, 0.06145433890903323, 0.21692087262775184, 0.10711847762830007, 0.056580668183153285, 0.16068903458028022, 0.1734724297374788, 0.10252654057406789, 0.08341402052897087, 

In [27]:
import pandas as pd

# Assuming `data` is already a pandas DataFrame with the necessary columns
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)
data['prev_local_max'] = data['high'].shift(1)

def calculate_reversions_and_profit(data, threshold_percentage, stop_loss_percentage=0.1):
    move_out_count = 0
    reversion_count = 0
    local_max_reach_count = 0
    reversion_after_local_max = 0
    currently_outside = False
    reached_local_max = False
    reversion_gap = []

    total_profit = 0
    total_target_hit_count = 0
    total_stop_loss_count = 0
    total_initial_position = 0  # To track the total initial position amount for percentage calculation
    daily_profits = {}
    daily_initial_positions = {}

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']
        prev_val = data.loc[i, 'prev_val']
        prev_local_max = data.loc[i, 'prev_local_max']
        current_date = data.loc[i, 'datetime'].date()

        if prev_vah is not None and prev_val is not None and prev_local_max is not None:
            threshold = prev_vah * threshold_percentage
            target_price = prev_val
            stop_loss_price = prev_local_max * (1 + stop_loss_percentage)

            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
                    reached_local_max = False
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False
                    if reached_local_max:
                        reversion_after_local_max += 1
                        gap = (prev_local_max - prev_vah) / prev_vah
                        reversion_gap.append(gap)
                        reached_local_max = False

            if close_price >= prev_local_max and currently_outside and not reached_local_max:
                local_max_reach_count += 1
                reached_local_max = True

                # Simulate the short position
                initial_position = prev_local_max
                total_initial_position += initial_position
                daily_initial_positions.setdefault(current_date, 0)
                daily_initial_positions[current_date] += initial_position
                position_opened = True
                for j in range(i+1, len(data)):
                    future_close_price = data.loc[j, 'close']
                    future_date = data.loc[j, 'datetime'].date()
                    if future_close_price <= target_price:
                        profit = prev_local_max - target_price
                        total_profit += profit
                        total_target_hit_count += 1
                        daily_profits.setdefault(current_date, 0)
                        daily_profits[current_date] += profit
                        position_opened = False
                        break
                    elif future_close_price >= stop_loss_price:
                        loss = stop_loss_price - prev_local_max
                        total_profit -= loss
                        total_stop_loss_count += 1
                        daily_profits.setdefault(current_date, 0)
                        daily_profits[current_date] -= loss
                        position_opened = False
                        break

                if position_opened:
                    future_close_price = data.loc[j, 'close']
                    profit_or_loss = prev_local_max - future_close_price
                    total_profit += profit_or_loss
                    daily_profits.setdefault(current_date, 0)
                    daily_profits[current_date] += profit_or_loss

            if current_date != data.loc[i - 1, 'datetime'].date():
                currently_outside = False
                reached_local_max = False

    average_reversion_gap = (sum(reversion_gap) / len(reversion_gap)) * 100 if reversion_gap else None
    total_profit_percentage = (total_profit / total_initial_position) * 100 if total_initial_position != 0 else None
    daily_profit_percentages = {date: (profit / daily_initial_positions[date]) * 100
                                for date, profit in daily_profits.items()}

    return {
        'move_out_count': move_out_count,
        'reversion_count': reversion_count,
        'local_max_reach_count': local_max_reach_count,
        'reversion_after_local_max': reversion_after_local_max,
        'average_reversion_gap': average_reversion_gap,
        'total_profit_percentage': total_profit_percentage,
        'total_target_hit_count': total_target_hit_count,
        'total_stop_loss_count': total_stop_loss_count,
        'daily_profit_percentages': daily_profit_percentages
    }

#thresholds = [0.0005, 0.00075, 0.001, 0.0015]
thresholds = [0.0005]
results = {}
for threshold in thresholds:
    stats = calculate_reversions_and_profit(data, threshold)
    move_out_count = stats['move_out_count']
    reversion_count = stats['reversion_count']
    local_max_reach_count = stats['local_max_reach_count']
    reversion_after_local_max = stats['reversion_after_local_max']
    average_reversion_gap = stats['average_reversion_gap']
    total_profit_percentage = stats['total_profit_percentage']
    total_target_hit_count = stats['total_target_hit_count']
    total_stop_loss_count = stats['total_stop_loss_count']
    daily_profit_percentages = stats['daily_profit_percentages']

    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count,
        'Reached local max': local_max_reach_count,
        'Reverted after reaching local max': reversion_after_local_max,
        'Average reversion gap (%)': average_reversion_gap,
        'Total Profit (%)': total_profit_percentage,
        'Total Target Hit Count': total_target_hit_count,
        'Total Stop Loss Count': total_stop_loss_count,
        'Daily Profit Percentages (%)': daily_profit_percentages
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")
    print(f"  Reached local max: {result['Reached local max']}")
    print(f"  Reverted after reaching local max: {result['Reverted after reaching local max']}")
    print(f"  Average reversion gap: {result['Average reversion gap (%)']:.2f}%")
    print(f"  Total Profit: {result['Total Profit (%)']:.2f}%")
    print(f"  Total Target Hit Count: {result['Total Target Hit Count']}")
    print(f"  Total Stop Loss Count: {result['Total Stop Loss Count']}")
    print("  Daily Profit Percentages:")
    for date, profit_percentage in result['Daily Profit Percentages (%)'].items():
        print(f"    {date}: {profit_percentage:.2f}%")

# Suggestions to reduce stop-loss hits:
# 1. Adjust the stop-loss percentage: You may try different values for `stop_loss_percentage` to find a more optimal value.
# 2. Trailing stop-loss: Implementing a trailing stop-loss could help in protecting profits while allowing the trade to continue if the price moves favorably.
# 3. Additional technical indicators: Use indicators like moving averages, RSI, or MACD to confirm the entry signals before taking a position.


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
  Reached local max: 439
  Reverted after reaching local max: 144
  Average reversion gap: 0.15%
  Total Profit: -0.81%
  Total Target Hit Count: 367
  Total Stop Loss Count: 67
  Daily Profit Percentages:
    2018-01-05: 0.24%
    2018-01-08: 0.37%
    2018-01-12: 0.26%
    2018-01-15: 0.23%
    2018-01-18: 0.68%
    2018-01-23: 0.54%
    2018-02-23: 0.20%
    2018-02-26: 0.87%
    2018-02-27: 1.02%
    2018-02-28: 0.64%
    2018-03-06: 0.39%
    2018-03-27: -10.00%
    2018-04-10: 0.12%
    2018-04-13: 0.45%
    2018-04-27: -10.00%
    2018-04-30: -10.00%
    2018-05-07: 0.18%
    2018-05-08: 0.43%
    2018-05-10: 0.34%
    2018-05-14: 0.92%
    2018-05-15: 0.74%
    2018-05-25: 0.95%
    2018-05-28: 0.60%
    2018-05-30: 0.87%
    2018-05-31: 0.76%
    2018-06-01: 0.26%
    2018-06-06: 0.59%
    2018-06-07: 0.42%
    2018-06-11: 0.32%
    2018-07-09: 0.03%
    2018-07-10: 0.21%
    2018-07-12: 0.03%
    2018-07-18: 0.88

# Higher POC

In [ ]:
'''import pandas as pd
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data.sort_values(by='datetime', inplace=True)
daily_data = data.groupby('date').agg({
    'poc': 'first',  # POC is the same for the entire day, taking the first occurrence
}).reset_index()

daily_data['poc_change'] = daily_data['poc'].pct_change()
daily_data['poc_trending_higher'] = (
    (daily_data['poc'] > daily_data['poc'].shift(1) * 1.002) &
    (daily_data['poc'].shift(1) > daily_data['poc'].shift(2) * 1.002)
)

trending_higher_dates = daily_data[daily_data['poc_trending_higher']]['date']
results = []

def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()
near_ranges = [0.01,0.015, 0.02, 0.025]  # 1.5%, 2%, 2.5%

for date in trending_higher_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_min = prev_day_data['close'].min()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            if ((daily_subset['close'] <= local_min * (1 + near_range)) &
                (daily_subset['close'] >= local_min * (1 - near_range))).any():

                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = ((last_5min_close - local_min) / local_min) * 100
                    results.append({'date': date, 'near_range': near_range, 'movement_pct': movement_pct})

results_df = pd.DataFrame(results)
summary_stats = results_df.groupby('near_range')['movement_pct'].describe()
#print(summary_stats)
#print(results_df.head())
total_unique_dates = results_df.groupby('near_range')['date'].nunique()
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()
bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)
print("Total Unique Number of Dates:")
print(total_unique_dates)
print("Total Percentage Change:")
print(total_pct_change)
print("Movement Percentage Range Counts:")
print(range_counts)
'''

Total Unique Number of Dates:
near_range
0.010    128
0.015    182
0.020    204
0.025    220
Name: date, dtype: int64
Total Percentage Change:
near_range
0.010    163.568467
0.015    286.174179
0.020    349.437409
0.025    403.065061
Name: movement_pct, dtype: float64
Movement Percentage Range Counts:
movement_range  <-2%  -2%-0%  0%-2%  >2%
near_range                              
0.010              0       6     99   23
0.015              0       6    124   52
0.020              0       6    129   69
0.025              0       6    129   85


In [ ]:
import pandas as pd
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data.sort_values(by='datetime', inplace=True)

daily_data = data.groupby('date').agg({
    'poc': 'first',  # POC is the same for the entire day, taking the first occurrence
}).reset_index()
daily_data['poc_change'] = daily_data['poc'].pct_change()

# Identify periods where daily POC is higher than the previous two days,
# and the increase is at least 0.2% each day
daily_data['poc_trending_higher'] = (
    (daily_data['poc'] > daily_data['poc'].shift(1) * 1.001) &
    (daily_data['poc'].shift(1) > daily_data['poc'].shift(2) * 1.001))

trending_higher_dates = daily_data[daily_data['poc_trending_higher']]['date']
results = []
def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()
near_ranges = [0.01, 0.015, 0.02, 0.025]  # 1%, 1.5%, 2%, 2.5%

for date in trending_higher_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_min = prev_day_data['close'].min()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            within_range = daily_subset[
                (daily_subset['close'] <= local_min * (1 + near_range)) &
                (daily_subset['close'] >= local_min * (1 - near_range))
            ]
            if not within_range.empty:
                entry_close = within_range['close'].min()
                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = ((last_5min_close - entry_close) / entry_close) * 100

                    results.append({
                        'date': date,
                        'near_range': near_range,
                        'movement_pct': movement_pct,
                        'entry_close': entry_close,
                        'last_5min_close': last_5min_close
                    })

results_df = pd.DataFrame(results)
summary_stats = results_df.groupby('near_range')['movement_pct'].describe()

'''
print(summary_stats)
print(results_df.head())'''

total_unique_dates = results_df.groupby('near_range')['date'].nunique()
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()
bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)

print("TOTAL UNIQUE NUMBER OF DATES:")
print(total_unique_dates)
print(" ")
print("TOTAL PERCENTAGE CHANGE:")
print(total_pct_change)
print(" ")
print("MOVEMENT PERCENTAGE RANGE COUNTS:")
print(range_counts)


TOTAL UNIQUE NUMBER OF DATES:
near_range
0.010    167
0.015    227
0.020    252
0.025    270
Name: date, dtype: int64
 
TOTAL PERCENTAGE CHANGE:
near_range
0.010    134.645266
0.015    192.780489
0.020    218.593417
0.025    235.440886
Name: movement_pct, dtype: float64
 
MOVEMENT PERCENTAGE RANGE COUNTS:
movement_range  <-2%  -2%-0%  0%-2%  >2%
near_range                              
0.010              0      10    143   14
0.015              0       9    198   20
0.020              0      10    219   23
0.025              0      10    233   27


In [ ]:
import pandas as pd
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data.sort_values(by='datetime', inplace=True)
daily_data = data.groupby('date').agg({
    'poc': 'first',
}).reset_index()

daily_data['poc_change'] = daily_data['poc'].pct_change()

daily_data['poc_trending_lower'] = (
    (daily_data['poc'] < daily_data['poc'].shift(1) * 0.998) &
    (daily_data['poc'].shift(1) < daily_data['poc'].shift(2) * 0.998)
)
trending_lower_dates = daily_data[daily_data['poc_trending_lower']]['date']

results = []

def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()
near_ranges = [0.01, 0.015, 0.02, 0.025]  # 1%, 1.5%, 2%, 2.5%

for date in trending_lower_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_max = prev_day_data['close'].max()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            within_range = daily_subset[
                (daily_subset['close'] <= local_max * (1 + near_range)) &
                (daily_subset['close'] >= local_max * (1 - near_range))
            ]
            if not within_range.empty:
                entry_close = within_range['close'].max()

                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = -((last_5min_close - entry_close) / entry_close) * 100
                    results.append({
                        'date': date,
                        'near_range': near_range,
                        'movement_pct': movement_pct,
                        'entry_close': entry_close,
                        'last_5min_close': last_5min_close
                    })

results_df = pd.DataFrame(results)
summary_stats = results_df.groupby('near_range')['movement_pct'].describe()
total_unique_dates = results_df.groupby('near_range')['date'].nunique()
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()
bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)

print("TOTAL UNIQUE NUMBER OF DATES:")
print(total_unique_dates)
print(" ")
print("TOTAL PERCENTAGE CHANGE:")
print(total_pct_change)
print(" ")
print("MOVEMENT PERCENTAGE RANGE COUNTS:")
print(range_counts)


TOTAL UNIQUE NUMBER OF DATES:
near_range
0.010    106
0.015    142
0.020    162
0.025    178
Name: date, dtype: int64
 
TOTAL PERCENTAGE CHANGE:
near_range
0.010    112.635941
0.015    147.173047
0.020    177.286408
0.025    199.366136
Name: movement_pct, dtype: float64
 
MOVEMENT PERCENTAGE RANGE COUNTS:
movement_range  <-2%  -2%-0%  0%-2%  >2%
near_range                              
0.010              0       7     83   16
0.015              0       9    111   22
0.020              0       9    125   28
0.025              0       9    138   31


In [ ]:

data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data.sort_values(by='datetime', inplace=True)
daily_data = data.groupby('date').agg({
    'poc': 'first',
}).reset_index()

daily_data['poc_change'] = daily_data['poc'].pct_change()

daily_data['poc_trending_higher'] = (
    (daily_data['poc'] > daily_data['poc'].shift(1) * 1.002) &
    (daily_data['poc'].shift(1) > daily_data['poc'].shift(2) * 1.002)
)
daily_data['poc_trending_lower'] = (
    (daily_data['poc'] < daily_data['poc'].shift(1) * 0.998) &
    (daily_data['poc'].shift(1) < daily_data['poc'].shift(2) * 0.998)
)

trending_higher_dates = daily_data[daily_data['poc_trending_higher']]['date']
trending_lower_dates = daily_data[daily_data['poc_trending_lower']]['date']

results = []

def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()
near_ranges = [0.01, 0.015, 0.02, 0.025]  # 1%, 1.5%, 2%, 2.5%

for date in trending_higher_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_min = prev_day_data['close'].min()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            within_range = daily_subset[
                (daily_subset['close'] <= local_min * (1 + near_range)) &
                (daily_subset['close'] >= local_min * (1 - near_range))
            ]
            if not within_range.empty:
                entry_close = within_range['close'].min()
                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = ((last_5min_close - entry_close) / entry_close) * 100

                    results.append({
                        'date': date,
                        'near_range': near_range,
                        'movement_pct': movement_pct,
                        'entry_close': entry_close,
                        'last_5min_close': last_5min_close,
                        'strategy': 'Higher High POC'
                    })

for date in trending_lower_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_max = prev_day_data['close'].max()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            within_range = daily_subset[
                (daily_subset['close'] <= local_max * (1 + near_range)) &
                (daily_subset['close'] >= local_max * (1 - near_range))
            ]
            if not within_range.empty:
                entry_close = within_range['close'].max()
                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = -((last_5min_close - entry_close) / entry_close) * 100

                    results.append({
                        'date': date,
                        'near_range': near_range,
                        'movement_pct': movement_pct,
                        'entry_close': entry_close,
                        'last_5min_close': last_5min_close,
                        'strategy': 'Lower Low POC'
                    })


results_df = pd.DataFrame(results)

summary_stats = results_df.groupby('near_range')['movement_pct'].describe()
total_unique_dates = results_df.groupby('near_range')['date'].nunique()
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()

bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)

print("TOTAL UNIQUE NUMBER OF DATES:")
print(total_unique_dates)
print(" ")
print("TOTAL PERCENTAGE CHANGE:")
print(total_pct_change)
print(" ")
print("MOVEMENT PERCENTAGE RANGE COUNTS:")
print(range_counts)


TOTAL UNIQUE NUMBER OF DATES:
near_range
0.010    234
0.015    324
0.020    366
0.025    398
Name: date, dtype: int64
 
TOTAL PERCENTAGE CHANGE:
near_range
0.010    224.440184
0.015    313.968347
0.020    368.342366
0.025    406.940117
Name: movement_pct, dtype: float64
 
MOVEMENT PERCENTAGE RANGE COUNTS:
movement_range  <-2%  -2%-0%  0%-2%  >2%
near_range                              
0.010              0      14    191   29
0.015              0      15    269   40
0.020              0      16    301   49
0.025              0      16    326   56


# Local Max-Minima

In [ ]:
df=data_.copy()
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

df['date'] = df.index.date
local_max_min = df.groupby('date')[['local_max', 'local_min']].first().shift(1)
df = df.merge(local_max_min, left_on='date', right_index=True, suffixes=('', '_prev'))

def check_rebound_fall(row):
    if pd.notna(row['local_max_prev']) and row['close'] >= row['local_max_prev']:
        return 'fall'
    elif pd.notna(row['local_min_prev']) and row['close'] <= row['local_min_prev']:
        return 'rebound'
    return None

df['signal'] = df.apply(check_rebound_fall, axis=1)
def calculate_movement(df):
    last_close = df['close'].iloc[-1]
    df['percentage_movement'] = df.apply(lambda row: ((last_close - row['close']) / row['close']) * 100 if row['signal'] else None, axis=1)
    return df

df = df.groupby('date').apply(calculate_movement)
df.reset_index(drop=True, inplace=True)
result = df[df['signal'].notna()][['date', 'close', 'signal', 'percentage_movement']]
result.head()

In [ ]:
signal_counts = result['signal'].value_counts()
total_movement = result.groupby('signal')['percentage_movement'].sum()
movement_value_counts = result.groupby('signal')['percentage_movement'].apply(lambda x: pd.cut(x, bins=4).value_counts())
signal_counts, total_movement, movement_value_counts

In [ ]:
df=data_
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

df['date'] = df.index.date
local_max_min = df.groupby('date')[['local_max', 'local_min']].first().shift(1)
df = df.merge(local_max_min, left_on='date', right_index=True, suffixes=('', '_prev'))

def check_rebound_fall(row):
    if pd.notna(row['local_max_prev']) and row['close'] >= row['local_max_prev']:
        return 'fall'
    elif pd.notna(row['local_min_prev']) and row['close'] <= row['local_min_prev']:
        return 'rebound'
    return None

df['signal'] = df.apply(check_rebound_fall, axis=1)
df = df[df['signal'].notna()]
first_occurrence_per_day = df.groupby('date').first().reset_index()

def calculate_movement(row, last_closes):
    last_close = last_closes[row['date']]
    return ((last_close - row['close']) / row['close']) * 100

last_closes = df.groupby('date')['close'].last().to_dict()
first_occurrence_per_day['percentage_movement'] = first_occurrence_per_day.apply(calculate_movement, last_closes=last_closes, axis=1)

result = first_occurrence_per_day[['date', 'close', 'signal', 'percentage_movement']]
signal_counts = result['signal'].value_counts()
total_movement = result.groupby('signal')['percentage_movement'].sum()
movement_value_counts = result.groupby('signal')['percentage_movement'].apply(lambda x: pd.cut(x, bins=4).value_counts())
signal_counts, total_movement, movement_value_counts

(signal
 fall       512
 rebound    423
 Name: count, dtype: int64,
 signal
 fall       105.533789
 rebound   -100.856700
 Name: percentage_movement, dtype: float64,
 signal                    
 fall     (-3.192, 0.17]       278
          (0.17, 3.532]        221
          (-6.568, -3.192]       7
          (3.532, 6.895]         6
 rebound  (-2.281, 2.004]      401
          (-6.566, -2.281]      14
          (2.004, 6.289]         7
          (-10.867, -6.566]      1
 Name: percentage_movement, dtype: int64)

# VWAP REJECTION

In [ ]:
# Assuming you have a DataFrame named 'data' containing your dataset with columns 'close', 'vwap', and 'max_pain'
data=data_
# Define a threshold for closely aligned Max Pain and VWAP
alignment_threshold = 0.02  # For example, within 5%

# Initialize variables to track short opportunities and the current rejection movement
short_opportunities = 0
in_rejection = False

# Loop through each row in the DataFrame starting from the second row
for i in range(1, len(data)):
    close_price_previous = data.iloc[i - 1]['close']
    close_price_current = data.iloc[i]['close']
    vwap_price_current = data.iloc[i]['vwap']
    max_pain_price_current = data.iloc[i]['max_pain_strike']

    # Check if the current close price is approaching VWAP from below and then reversing
    if close_price_previous < vwap_price_current and close_price_current > vwap_price_current:
        # Check if Max Pain and VWAP are closely aligned
        if abs(max_pain_price_current - vwap_price_current) / vwap_price_current <= alignment_threshold:
            if not in_rejection:
                in_rejection = True
                short_opportunities += 1
    else:
        # Reset the rejection movement if the previous candle close is broken
        in_rejection = False

# Print the number of short opportunities identified
print("Number of short opportunities:", short_opportunities)


Number of short opportunities: 257


In [ ]:
# Assuming you have a DataFrame named 'data' containing your dataset with columns 'close' and 'vwap'

# Initialize variables to track the total number of approaches to VWAP and the number of rejected instances
approaches_to_vwap = 0
rejected_instances = 0

# Initialize variables to track the current rejection movement
in_rejection = False

# Loop through each row in the DataFrame starting from the second row
for i in range(1, len(data)):
    close_price_previous = data.iloc[i - 1]['close']
    close_price_current = data.iloc[i]['close']
    vwap_price_current = data.iloc[i]['vwap']

    # Check if the current close price is approaching VWAP
    if close_price_previous < vwap_price_current and close_price_current > vwap_price_current:
        approaches_to_vwap += 1

        # Set the rejection threshold as 70% of the rise from the approach to the rejection point
        rejection_threshold = close_price_current - close_price_previous * 0.7

        # Check if the rejection is successful
        for j in range(i, len(data)):
            if data.iloc[j]['close'] <= rejection_threshold:
                rejected_instances += 1
                break

# Print the total number of approaches to VWAP and the number of rejected instances
print("Total approaches to VWAP:", approaches_to_vwap)
print("Number of rejected instances:", rejected_instances)


Total approaches to VWAP: 277
Number of rejected instances: 0


# VWAP - II

In [ ]:
data=data_.copy()

# Define alignment range
alignment_range = 0.015

# Initialize variables
short_entry_price = None
total_profitable_trades = 0
profit_thresholds = [0,0.001,0.0015,0.002,0.0025, 0.005, 0.01, 0.015, 0.02, 0.025]
profit_counts = {threshold: 0 for threshold in profit_thresholds}
data['previous_day_local_min'] = data['local_min'].shift(1)
# Implement strategy
for index, row in data.iterrows():
    if abs(row['max_pain_strike'] - row['vwap']) <= row['vwap'] * alignment_range:
        if short_entry_price is None:
            short_entry_price = row['close']
    elif short_entry_price is not None:
        # Exit the short position if the price returns to the original position,
        # goes near the previous day's local minimum, or until the last 5-minute close
        if (row['close'] <= short_entry_price or
            row['close'] <= row['previous_day_local_min'] or
            index == len(data) - 1):
            exit_price = row['close']
            profit = (short_entry_price - exit_price) / short_entry_price
            if profit > 0:
                total_profitable_trades += 1
                for threshold in profit_thresholds:
                    if profit >= threshold:
                        profit_counts[threshold] += 1
            short_entry_price = None

# Output results
print("Total profitable trades:", total_profitable_trades)
for threshold in profit_thresholds:
    print(f"Profit >= {threshold * 100}%: {profit_counts[threshold]} times")

Total profitable trades: 437
Profit >= 0%: 437 times
Profit >= 0.1%: 217 times
Profit >= 0.15%: 153 times
Profit >= 0.2%: 118 times
Profit >= 0.25%: 95 times
Profit >= 0.5%: 37 times
Profit >= 1.0%: 20 times
Profit >= 1.5%: 11 times
Profit >= 2.0%: 7 times
Profit >= 2.5%: 4 times


In [ ]:
import pandas as pd

# Read the data
data = data_.copy()

# Define alignment range
alignment_range = 0.035

# Initialize variables
short_entry_price = None
total_profitable_trades = 0
total_profit = 0  # Initialize total profit variable
profit_thresholds = [0, 0.001, 0.0015, 0.002, 0.0025, 0.005, 0.01, 0.015, 0.02, 0.025]
profit_counts = {threshold: 0 for threshold in profit_thresholds}
data['previous_day_local_min'] = data['local_min'].shift(1)

# Implement strategy
for index, row in data.iterrows():
    if abs(row['max_pain_strike'] - row['vwap']) <= row['vwap'] * alignment_range:
        if short_entry_price is None:
            short_entry_price = row['close']
    elif short_entry_price is not None:
        # Exit the short position if the price returns to the original position,
        # goes near the previous day's local minimum, or until the last 5-minute close
        if (row['close'] <= short_entry_price or
                row['close'] <= row['previous_day_local_min'] or
                index == len(data) - 1):
            exit_price = row['close']
            profit = (short_entry_price - exit_price) / short_entry_price
            if profit > 0:
                total_profitable_trades += 1
                total_profit += profit  # Add profit to total profit
                for threshold in profit_thresholds:
                    if profit >= threshold:
                        profit_counts[threshold] += 1
            short_entry_price = None

# Output results
print("Total profitable trades:", total_profitable_trades)
print("Total profit made:", total_profit*100,"%")
for threshold in profit_thresholds:
    print(f"Profit >= {threshold * 100}%: {profit_counts[threshold]} times")


Total profitable trades: 293
Total profit made: 120.37722812060605 %
Profit >= 0%: 293 times
Profit >= 0.1%: 158 times
Profit >= 0.15%: 134 times
Profit >= 0.2%: 97 times
Profit >= 0.25%: 80 times
Profit >= 0.5%: 49 times
Profit >= 1.0%: 25 times
Profit >= 1.5%: 18 times
Profit >= 2.0%: 15 times
Profit >= 2.5%: 13 times


In [ ]:
import pandas as pd

# Read the data
data = data_.copy()

# Define alignment range
alignment_range = 0.015

# Initialize variables
short_entry_price = None
total_profitable_trades = 0
total_profit = 0  # Initialize total profit variable
profit_thresholds = [0, 0.001, 0.0015, 0.002, 0.0025, 0.005, 0.01, 0.015, 0.02, 0.025]
profit_counts = {threshold: 0 for threshold in profit_thresholds}
data['previous_day_local_max'] = data['local_max'].shift(1)
data['previous_day_local_min'] = data['local_min'].shift(1)
# Implement strategy
for index, row in data.iterrows():
    # Short selling when Max Pain and VWAP are aligned and PCR > 0.7
    if abs(row['max_pain_strike'] - row['vwap']) <= row['vwap'] * alignment_range and row['intraday_pcr'] < 0.7:
        if short_entry_price is None:
            short_entry_price = row['close']
    elif short_entry_price is not None:
        # Exit the short position if the price returns to the original position,
        # goes near the previous day's local minimum, or until the last 5-minute close
        if (row['close'] <= short_entry_price or
                row['close'] <= row['previous_day_local_min'] or
                index == len(data) - 1):
            exit_price = row['close']
            profit = (short_entry_price - exit_price) / short_entry_price
            if profit > 0:
                total_profitable_trades += 1
                total_profit += profit  # Add profit to total profit
                for threshold in profit_thresholds:
                    if profit >= threshold:
                        profit_counts[threshold] += 1
            short_entry_price = None

    # Buying when Max Pain and VWAP are aligned and PCR < 0.7
    elif abs(row['max_pain_strike'] - row['vwap']) <= row['vwap'] * alignment_range and row['intraday_pcr'] > 0.7:
        # Enter buy position
        # Logic to enter buy position here
        pass  # Placeholder for buy logic

# Output results
print("Total profitable trades:", total_profitable_trades)
print("Total profit made:", total_profit * 100, "%")
for threshold in profit_thresholds:
    print(f"Profit >= {threshold * 100}%: {profit_counts[threshold]} times")


Total profitable trades: 606
Total profit made: 94.20373531712086 %
Profit >= 0%: 606 times
Profit >= 0.1%: 251 times
Profit >= 0.15%: 181 times
Profit >= 0.2%: 132 times
Profit >= 0.25%: 92 times
Profit >= 0.5%: 38 times
Profit >= 1.0%: 9 times
Profit >= 1.5%: 5 times
Profit >= 2.0%: 3 times
Profit >= 2.5%: 1 times


# Higher Profiles(done)

In [ ]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

# Function to calculate the percentage of price movement from the opening price to the average of VAH and VAL
def calculate_price_movement(open_price, average_vah_val):
    return ((average_vah_val - open_price) / open_price) * 100

# Step 1: Check VAH trend for the previous two days
def is_vah_trending_up(data):
    return data.iloc[-1]['vah'] > data.iloc[-2]['vah']

# Step 2: Check if the first 5-minute candle opens near or below the previous day's VAL
def is_opening_near_or_below_val(open_price, val):
    return open_price <= val

# Step 3: Check if the price at any point moves up to the average of previous day's VAH and VAL
def check_reversal_to_range(df_day, val, average_vah_val):
    return df_day['close'].max() >= average_vah_val

# Applying the logic to the dataset
results = []
for i in range(2, len(df) - 1):
    # Check if the VAH is trending up for the previous two days
    if is_vah_trending_up(df.iloc[i-2:i]):
        # Get the VAL and VAH from the previous day
        previous_day_val = df.iloc[i-1]['val']
        previous_day_vah = df.iloc[i-1]['vah']
        average_vah_val = (previous_day_vah + previous_day_val) / 2

        # Get the first 5-minute candle data of the current day
        first_5min_candle = df.iloc[i:i+1]

        # Check if the first 5-minute candle opens near or below the previous day's VAL
        if is_opening_near_or_below_val(first_5min_candle.iloc[0]['open'], previous_day_val):
            # Get the data for the current day
            df_day = df[df['datetime'].dt.date == first_5min_candle.iloc[0]['datetime'].date()]
            # Check if the price at any point moves up to the average of the previous day's VAH and VAL
            if check_reversal_to_range(df_day, previous_day_val, average_vah_val):
                # Calculate the percentage of price movement
                price_movement_percentage = calculate_price_movement(first_5min_candle.iloc[0]['open'], average_vah_val)
                results.append(price_movement_percentage)

# Categorize the results into specified percentage ranges
percentage_ranges = {
    'less than 1%': 0,
    '1-1.5%': 0,
    '1.5-2%': 0,
    'greater than 2%': 0
}

for result in results:
    if result < 1:
        percentage_ranges['less than 1%'] += 1
    elif 1 <= result < 1.5:
        percentage_ranges['1-1.5%'] += 1
    elif 1.5 <= result < 2:
        percentage_ranges['1.5-2%'] += 1
    else:
        percentage_ranges['greater than 2%'] += 1

# Output the results
print("Percentage of price movement from opening price to average of VAH and VAL:")
for key, value in percentage_ranges.items():
    print(f"{key}: {value} ({(value / len(results)) * 100:.2f}%)")

Percentage of price movement from opening price to average of VAH and VAL:
less than 1%: 219 (71.10%)
1-1.5%: 45 (14.61%)
1.5-2%: 23 (7.47%)
greater than 2%: 21 (6.82%)


**VAH HIGHER**

In [11]:
import pandas as pd

df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

def calculate_price_movement(open_price, target_price):
    return ((target_price - open_price) / open_price) * 100

# Step 1: Check VAH trend for the previous two days
def is_vah_trending_up(data):
    return data.iloc[-1]['vah'] > data.iloc[-2]['vah']

# Step 2: Check if the first 5-minute candle opens near or below the previous day's VAL
def is_opening_near_or_below_val(open_price, val):
    return open_price <= val

# Initialize variables to store results
total_profit_percentage = 0
total_stop_loss_percentage = 0
profit_daywise_percentage = {}
stop_loss_daywise_percentage = {}
days_traded = 0
profit_days = 0
stop_loss_days = 0

# Applying the logic to the dataset
for i in range(2, len(df) - 1):
    # Check if the VAH is trending up for the previous two days
    if is_vah_trending_up(df.iloc[i-2:i]):
        # Get the VAL and VAH from the previous day
        previous_day_val = df.iloc[i-1]['val']
        previous_day_vah = df.iloc[i-1]['vah']
        average_vah_val = (previous_day_vah + previous_day_val) / 2

        # Get the first 5-minute candle data of the current day
        first_5min_candle = df.iloc[i:i+1]

        # Check if the first 5-minute candle opens near or below the previous day's VAL
        if is_opening_near_or_below_val(first_5min_candle.iloc[0]['open'], previous_day_val):
            # Check if the gap between the average of VAH-VAL and the opening price of the first 5-minute candle of the third day is at least 1%
            price_gap_percentage = calculate_price_movement(first_5min_candle.iloc[0]['open'], average_vah_val)
            if price_gap_percentage >= 0.75:
                days_traded += 1  # Increment the count of days traded

                # Calculate entry price and expected exit price
                entry_price = first_5min_candle.iloc[0]['open']
                exit_price = average_vah_val
                profit_percentage = calculate_price_movement(entry_price, exit_price)

                # Get the data for the current day to check stop-loss
                df_day = df[df['datetime'].dt.date == first_5min_candle.iloc[0]['datetime'].date()]
                stop_loss = df.iloc[i-1]['local_min']

                trade_executed = False  # Flag to check if trade has been executed

                # Iterate through the 5-minute candles of the current day to check for profit or stop-loss hit
                for j in range(len(df_day)):
                    current_close = df_day.iloc[j]['close']

                    if current_close >= exit_price:
                        profit_days += 1
                        total_profit_percentage += profit_percentage
                        profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage
                        trade_executed = True
                        break
                    elif current_close <= stop_loss:
                        stop_loss_days += 1
                        loss_percentage = ((stop_loss - entry_price) / entry_price) * 100
                        total_stop_loss_percentage += loss_percentage
                        stop_loss_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = loss_percentage
                        trade_executed = True
                        break

                # If no trade was executed (no profit or stop-loss hit)
                if not trade_executed:
                    total_profit_percentage += profit_percentage
                    profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage

# Print total profit in terms of percentage
print("Total Profit (in terms of percentage):", total_profit_percentage)
print("Total Stop Loss (in terms of percentage):", total_stop_loss_percentage)

# Print profit day-wise and stop-loss day-wise in terms of percentage
print("\nProfit Day-wise (in terms of percentage):")
for date, profit_percentage in profit_daywise_percentage.items():
    print(f"{date}: {profit_percentage}%")

print("\nStop Loss Day-wise (in terms of percentage):")
for date, loss_percentage in stop_loss_daywise_percentage.items():
    print(f"{date}: {loss_percentage}%")

# Print count of days traded, profit days, and stop-loss days
print("\nTotal Days Traded:", days_traded)
print("Days with Profit:", profit_days)
print("Days with Stop Loss:", stop_loss_days)


Total Profit (in terms of percentage): 213.6648954087274
Total Stop Loss (in terms of percentage): -0.21528559275351566

Profit Day-wise (in terms of percentage):
2018-01-17: 0.8866387569934622%
2018-01-19: 0.9797957730828772%
2018-02-08: 1.2767333965402183%
2018-03-12: 0.8416721123999228%
2018-03-26: 1.503463476070529%
2018-04-27: 1.0126481584599416%
2018-05-15: 1.4303977272727273%
2018-06-20: 0.8000136657138628%
2018-06-22: 0.7824780834340881%
2018-07-26: 0.8701663561450177%
2018-10-01: 1.0456867535071763%
2018-10-10: 2.009799959587784%
2018-10-12: 0.762783959133974%
2018-10-16: 0.8486687548942835%
2018-10-29: 1.48327265261151%
2018-12-07: 0.764786976884728%
2018-12-12: 0.9620162356050628%
2019-03-05: 1.1345370472310963%
2019-03-12: 0.757011917128599%
2019-03-13: 1.7075489520397755%
2019-03-15: 0.9370201003310267%
2019-03-26: 1.2715336899555163%
2019-03-28: 0.898861130911926%
2019-05-14: 0.9462835727670106%
2019-05-17: 1.2989270257014822%
2019-05-20: 1.347671730515194%
2019-05-27: 0.

**Lower VAH**

In [8]:
import pandas as pd

# Assuming data_ is already provided
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

def calculate_price_movement(open_price, target_price):
    return ((open_price - target_price) / open_price) * 100

# Step 1: Check VAL trend for the previous two days
def is_val_trending_down(data):
    return data.iloc[-1]['vah'] < data.iloc[-2]['vah']

# Step 2: Check if the first 5-minute candle opens near or above the previous day's VAH
def is_opening_near_or_above_vah(open_price, vah):
    return open_price >= vah

# Initialize variables to store results
total_profit_percentage = 0
total_stop_loss_percentage = 0
profit_daywise_percentage = {}
stop_loss_daywise_percentage = {}
days_traded = 0
profit_days = 0
stop_loss_days = 0

# Applying the logic to the dataset
for i in range(2, len(df) - 1):
    # Check if the VAL is trending down for the previous two days
    if is_val_trending_down(df.iloc[i-2:i]):
        # Get the VAL and VAH from the previous day
        previous_day_val = df.iloc[i-1]['val']
        previous_day_vah = df.iloc[i-1]['vah']
        average_vah_val = (previous_day_vah + previous_day_val) / 2

        # Get the first 5-minute candle data of the current day
        first_5min_candle = df.iloc[i:i+1]

        # Check if the first 5-minute candle opens near or above the previous day's VAH
        if is_opening_near_or_above_vah(first_5min_candle.iloc[0]['open'], previous_day_vah):
            # Check if the gap between the average of VAH-VAL and the opening price of the first 5-minute candle of the third day is at least 1%
            price_gap_percentage = calculate_price_movement(first_5min_candle.iloc[0]['open'], average_vah_val)
            if price_gap_percentage >= 0.1:
                days_traded += 1  # Increment the count of days traded

                # Calculate entry price and expected exit price
                entry_price = first_5min_candle.iloc[0]['open']
                exit_price = average_vah_val
                profit_percentage = calculate_price_movement(entry_price, exit_price)

                # Get the data for the current day to check stop-loss
                df_day = df[df['datetime'].dt.date == first_5min_candle.iloc[0]['datetime'].date()]
                stop_loss = df.iloc[i-1]['local_max']

                trade_executed = False  # Flag to check if trade has been executed

                # Iterate through the 5-minute candles of the current day to check for profit or stop-loss hit
                for j in range(len(df_day)):
                    current_close = df_day.iloc[j]['close']

                    if current_close <= exit_price:
                        profit_days += 1
                        total_profit_percentage += profit_percentage
                        profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage
                        trade_executed = True
                        break
                    elif current_close >= stop_loss:
                        stop_loss_days += 1
                        loss_percentage = ((entry_price - stop_loss) / entry_price) * 100
                        total_stop_loss_percentage += loss_percentage
                        stop_loss_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = loss_percentage
                        trade_executed = True
                        break

                # If no trade was executed (no profit or stop-loss hit)
                if not trade_executed:
                    total_profit_percentage += profit_percentage
                    profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage

# Print total profit in terms of percentage
print("Total Profit (in terms of percentage):", total_profit_percentage)
print("Total Stop Loss (in terms of percentage):", total_stop_loss_percentage)

# Print profit day-wise and stop-loss day-wise in terms of percentage
print("\nProfit Day-wise (in terms of percentage):")
for date, profit_percentage in profit_daywise_percentage.items():
    print(f"{date}: {profit_percentage}%")

print("\nStop Loss Day-wise (in terms of percentage):")
for date, loss_percentage in stop_loss_daywise_percentage.items():
    print(f"{date}: {loss_percentage}%")

# Print count of days traded, profit days, and stop-loss days
print("\nTotal Days Traded:", days_traded)
print("Days with Profit:", profit_days)
print("Days with Stop Loss:", stop_loss_days)


Total Profit (in terms of percentage): 24.65667044232925
Total Stop Loss (in terms of percentage): -0.001617488283310421

Profit Day-wise (in terms of percentage):
2018-01-02: 0.2368689044690711%
2018-02-16: 0.6767324936909809%
2018-02-21: 0.20176836991134178%
2018-06-04: 0.9149094980406846%
2018-09-05: 0.3794506759218812%
2019-02-28: 0.17998513528005544%
2019-04-30: 0.2980622958178489%
2019-07-02: 0.31613698820967057%
2019-07-18: 0.4279452944279859%
2019-07-19: 1.1058934118031691%
2019-08-20: 0.45831543556539933%
2019-09-05: 0.5897359682410613%
2019-10-04: 1.3574572726691942%
2019-10-24: 0.9193061840120713%
2019-11-13: 0.6837053450100528%
2019-11-29: 0.22374379568280725%
2019-12-10: 0.28381079280479404%
2020-03-18: 3.8105307414552203%
2020-03-24: 3.6125590065930866%
2020-09-23: 0.5238437951276274%
2021-02-10: 0.6614861911439734%
2021-09-06: 0.3618781702642386%
2021-09-09: 0.22733256374425342%
2021-12-16: 0.5922495228496065%
2022-01-24: 1.1512674109614627%
2022-01-31: 0.292302682990352

In [15]:
import pandas as pd

df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

def calculate_price_movement(open_price, target_price):
    return ((open_price - target_price) / open_price) * 100

# Step 1: Check if VAL is trending down for the previous two consecutive days
def is_val_trending_down(data):
    return data.iloc[-1]['vah'] < data.iloc[-2]['vah']

# Step 2: Check if the first 5-minute candle opens near or above the previous day's VAH
def is_opening_near_or_above_vah(open_price, vah):
    return open_price >= vah

# Initialize variables to store results
total_profit_percentage = 0
total_stop_loss_percentage = 0
profit_daywise_percentage = {}
stop_loss_daywise_percentage = {}
days_traded = 0
profit_days = 0
stop_loss_days = 0

# Applying the logic to the dataset
for i in range(3, len(df) - 1):
    # Check if the VAL is trending down for the previous two consecutive days
    if is_val_trending_down(df.iloc[i-3:i]):
        # Get the VAL and VAH from the previous day
        previous_day_val = df.iloc[i-1]['val']
        previous_day_vah = df.iloc[i-1]['vah']
        average_vah_val = (previous_day_vah + previous_day_val) / 2

        # Get the first 5-minute candle data of the current day
        first_5min_candle = df.iloc[i:i+1]

        # Check if the first 5-minute candle opens near or above the previous day's VAH
        if is_opening_near_or_above_vah(first_5min_candle.iloc[0]['open'], previous_day_vah):
            # Check if the gap between the average of VAH-VAL and the opening price of the first 5-minute candle of the third day is at least 1%
            price_gap_percentage = calculate_price_movement(first_5min_candle.iloc[0]['open'], average_vah_val)
            if price_gap_percentage >= 1:
                days_traded += 1  # Increment the count of days traded

                # Calculate entry price and expected exit price
                entry_price = first_5min_candle.iloc[0]['open']
                exit_price = average_vah_val
                profit_percentage = calculate_price_movement(entry_price, exit_price)

                # Get the data for the current day to check stop-loss
                df_day = df[df['datetime'].dt.date == first_5min_candle.iloc[0]['datetime'].date()]
                stop_loss = df.iloc[i-1]['local_max']

                trade_executed = False  # Flag to check if trade has been executed

                # Iterate through the 5-minute candles of the current day to check for profit or stop-loss hit
                for j in range(len(df_day)):
                    current_close = df_day.iloc[j]['close']

                    if current_close <= exit_price:
                        profit_days += 1
                        total_profit_percentage += profit_percentage
                        profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage
                        trade_executed = True
                        break
                    elif current_close >= stop_loss:
                        stop_loss_days += 1
                        loss_percentage = ((entry_price - stop_loss) / entry_price) * 100
                        total_stop_loss_percentage += loss_percentage
                        stop_loss_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = loss_percentage
                        trade_executed = True
                        break

                # If no trade was executed (no profit or stop-loss hit)
                if not trade_executed:
                    total_profit_percentage += profit_percentage
                    profit_daywise_percentage[first_5min_candle.iloc[0]['datetime'].date()] = profit_percentage

# Print total profit in terms of percentage
print("Total Profit (in terms of percentage):", total_profit_percentage)
print("Total Stop Loss (in terms of percentage):", total_stop_loss_percentage)

# Print profit day-wise and stop-loss day-wise in terms of percentage
print("\nProfit Day-wise (in terms of percentage):")
for date, profit_percentage in profit_daywise_percentage.items():
    print(f"{date}: {profit_percentage}%")

print("\nStop Loss Day-wise (in terms of percentage):")
for date, loss_percentage in stop_loss_daywise_percentage.items():
    print(f"{date}: {loss_percentage}%")

# Print count of days traded, profit days, and stop-loss days
print("\nTotal Days Traded:", days_traded)
print("Days with Profit:", profit_days)
print("Days with Stop Loss:", stop_loss_days)


Total Profit (in terms of percentage): 14.685134846559494
Total Stop Loss (in terms of percentage): 0

Profit Day-wise (in terms of percentage):
2019-07-19: 1.1058934118031691%
2019-10-04: 1.3574572726691942%
2020-03-18: 3.8105307414552203%
2020-03-24: 3.6125590065930866%
2022-01-24: 1.1512674109614627%
2022-02-07: 1.0873233522047834%
2022-03-03: 1.127979171397163%
2022-06-16: 1.432124479475414%

Stop Loss Day-wise (in terms of percentage):

Total Days Traded: 8
Days with Profit: 8
Days with Stop Loss: 0


# VAH -VAL GAP FILL POC

In [ ]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Parameters
entry_price = None
exit_price = None
total_trades = 0
trades_hitting_profit = 0
trades_hitting_stoploss = 0
total_profit_percentage = 0
total_stoploss_percentage = 0

# Helper function to find the previous day with data
def get_previous_day_levels(df, current_date):
    previous_date = current_date - pd.Timedelta(days=1)
    while previous_date >= df.index.date[0]:
        previous_day = df[df.index.date == previous_date]
        if not previous_day.empty:
            vah = previous_day['vah'].iloc[0]
            val = previous_day['val'].iloc[0]
            poc = previous_day['poc'].iloc[0]
            max_pain = previous_day['max_pain_strike'].iloc[0]
            return vah, val, poc, max_pain
        previous_date -= pd.Timedelta(days=1)
    return None, None, None, None

# Iterate through the DataFrame, assuming the data is sorted by datetime
for current_date in df.index.date:
    vah, val, poc, max_pain = get_previous_day_levels(df, current_date)
    if vah is None:
        continue

    # Filter current day's data
    current_day = df[df.index.date == current_date]

    for idx, row in current_day.iterrows():
        open_price = row['open']
        high = row['high']
        low = row['low']
        close = row['close']

        if entry_price is None:
            # Check for entry signal
            if open_price > vah:
                entry_price = open_price
                target_price = poc
                stop_loss = val
                total_trades += 1
        else:
            # Check for exit conditions
            if high >= max_pain:
                exit_price = max_pain
                trades_hitting_profit += 1
                total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                break
            elif low <= stop_loss:
                exit_price = stop_loss
                trades_hitting_stoploss += 1
                total_stoploss_percentage += ((entry_price - exit_price) / entry_price) * 100
                break
            elif close >= poc:
                exit_price = poc
                trades_hitting_profit += 1
                total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                break

    if entry_price and exit_price:
        entry_price = None
        exit_price = None

# Output trade summary
print(f"Total Trades: {total_trades}")
print(f"Trades Hitting Profit: {trades_hitting_profit}")
print(f"Trades Hitting Stoploss: {trades_hitting_stoploss}")
print(f"Total Profit Percentage: {total_profit_percentage:.2f}%")
print(f"Total Stoploss Percentage: {total_stoploss_percentage:.2f}%")

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter data for the year 2018
df_2018 = df.loc[df['datetime'].dt.year == 2018]

# Parameters
total_trades = 0
trades_hitting_profit = 0
trades_hitting_stoploss = 0
total_profit_percentage = 0
total_stoploss_percentage = 0

# Iterate through the DataFrame groups
for date, group in df_2018.groupby(df_2018['datetime'].dt.date):
    entry_price = None
    exit_price = None

    # Get previous day's data
    previous_day = df_2018[df_2018['datetime'].dt.date == date - pd.Timedelta(days=1)]
    if previous_day.empty:
        continue

    vah = previous_day['vah'].iloc[0]
    val = previous_day['val'].iloc[0]
    poc = previous_day['poc'].iloc[0]
    max_pain = previous_day['max_pain_strike'].iloc[0]

    # Flag to track if entry point is found
    entry_found = False

    # Iterate through rows in the group
    for _, row in group.iterrows():
        close_price = row['close']
        high = row['high']
        low = row['low']
        close = row['close']

        if close_price > vah:
            entry_price = close_price
            target_price = poc
            stop_loss = val
            total_trades += 1
            entry_found = True

        if entry_found:
            # Check for exit conditions
            if high >= max_pain:
                exit_price = max_pain
                trades_hitting_profit += 1
                total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                break
            elif low <= stop_loss:
                exit_price = stop_loss
                trades_hitting_stoploss += 1
                total_stoploss_percentage += ((entry_price - exit_price) / entry_price) * 100
                break
            elif close >= poc:
                exit_price = poc
                trades_hitting_profit += 1
                total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                break

    # If the loop completes without exiting the position, consider it closed at the close price
    if entry_found and exit_price is None:
        exit_price = close
        trades_hitting_profit += 1
        total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100

# Output trade summary
print(f"Total Trades: {total_trades}")
print(f"Trades Hitting Profit: {trades_hitting_profit}")
print(f"Trades Hitting Stoploss: {trades_hitting_stoploss}")
print(f"Total Profit Percentage: {total_profit_percentage:.2f}%")
print(f"Total Stoploss Percentage: {total_stoploss_percentage:.2f}%")


Total Trades: 88
Trades Hitting Profit: 88
Trades Hitting Stoploss: 0
Total Profit Percentage: -124.12%
Total Stoploss Percentage: 0.00%


In [ ]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])

# Filter data for the year 2018
df_2018 = df.loc[df['datetime'].dt.year == 2018]

# Parameters
total_trades = 0
trades_hitting_profit = 0
trades_hitting_stoploss = 0
total_profit_percentage = 0
total_stoploss_percentage = 0

# Iterate through the DataFrame groups
for date, group in df_2018.groupby(df_2018['datetime'].dt.date):
    entry_price = None
    exit_price = None

    # Get previous day's data
    previous_day = df_2018[df_2018['datetime'].dt.date == date - pd.Timedelta(days=1)]
    if previous_day.empty:
        continue

    vah = previous_day['vah'].iloc[0]
    val = previous_day['val'].iloc[0]
    poc = previous_day['poc'].iloc[0]
    max_pain = previous_day['max_pain_strike'].iloc[0]

    # Flag to track if entry point is found
    entry_found = False
    long_trade = False

    # Iterate through rows in the group
    for _, row in group.iterrows():
        close_price = row['close']
        high = row['high']
        low = row['low']
        open_price = row['open']

        # Check for gap up or gap down scenarios
        if entry_price is None:
            if open_price > vah:
                # Gap up, consider long trade if POC is above
                if poc > open_price:
                    entry_price = close_price
                    target_price = poc
                    stop_loss = val
                    long_trade = True
                    entry_found = True
                    total_trades += 1
            elif open_price < val:
                # Gap down, consider short trade if POC is below
                if poc < open_price:
                    entry_price = close_price
                    target_price = poc
                    stop_loss = vah
                    long_trade = False
                    entry_found = True
                    total_trades += 1

        if entry_found:
            # Check for exit conditions for long trades
            if long_trade:
                if high >= max_pain:
                    exit_price = max_pain
                    trades_hitting_profit += 1
                    total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                    break
                elif low <= stop_loss:
                    exit_price = stop_loss
                    trades_hitting_stoploss += 1
                    total_stoploss_percentage += ((entry_price - exit_price) / entry_price) * 100
                    break
                elif close_price >= target_price:
                    exit_price = target_price
                    trades_hitting_profit += 1
                    total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
                    break
            # Check for exit conditions for short trades
            else:
                if low <= max_pain:
                    exit_price = max_pain
                    trades_hitting_profit += 1
                    total_profit_percentage += ((entry_price - exit_price) / entry_price) * 100
                    break
                elif high >= stop_loss:
                    exit_price = stop_loss
                    trades_hitting_stoploss += 1
                    total_stoploss_percentage += ((exit_price - entry_price) / entry_price) * 100
                    break
                elif close_price <= target_price:
                    exit_price = target_price
                    trades_hitting_profit += 1
                    total_profit_percentage += ((entry_price - exit_price) / entry_price) * 100
                    break

    # If the loop completes without exiting the position, consider it closed at the close price
    if entry_found and exit_price is None:
        exit_price = close_price
        if long_trade:
            trades_hitting_profit += 1
            total_profit_percentage += ((exit_price - entry_price) / entry_price) * 100
        else:
            trades_hitting_profit += 1
            total_profit_percentage += ((entry_price - exit_price) / entry_price) * 100

# Output trade summary
print(f"Total Trades: {total_trades}")
print(f"Trades Hitting Profit: {trades_hitting_profit}")
print(f"Trades Hitting Stoploss: {trades_hitting_stoploss}")
print(f"Total Profit Percentage: {total_profit_percentage:.2f}%")
print(f"Total Stoploss Percentage: {total_stoploss_percentage:.2f}%")

Total Trades: 29
Trades Hitting Profit: 26
Trades Hitting Stoploss: 3
Total Profit Percentage: -27.98%
Total Stoploss Percentage: 0.20%


In [ ]:
data=data_
bullish_trends = 0
bearish_trends = 0
moves_to_poc = 0

# Define the 1.5% threshold
threshold = 1.5 / 100

# Iterate through the dataset
for i in range(1, len(data)):
    previous_day_data = data.iloc[i-1]
    current_day_data = data.iloc[i]

    # Identify gap openings
    if current_day_data['open'] > previous_day_data['vah']:
        # Check for bullish trend
        if current_day_data['close'] >= previous_day_data['vah'] * (1 + threshold):
            bullish_trends += 1
    elif current_day_data['open'] < previous_day_data['val']:
        # Check for bearish trend
        if current_day_data['close'] <= previous_day_data['val'] * (1 - threshold):
            bearish_trends += 1

    # Check for move to POC
    if current_day_data['low'] <= current_day_data['poc'] <= current_day_data['high']:
        moves_to_poc += 1

# Output the results
print(f"Bullish Trends: {bullish_trends}")
print(f"Bearish Trends: {bearish_trends}")
print(f"Moves to POC: {moves_to_poc}")

Bullish Trends: 7
Bearish Trends: 241
Moves to POC: 11321


In [ ]:
data=data_
bullish_trends = 0
bearish_trends = 0
moves_to_poc = 0

# Define the 1.5% threshold
threshold = 1.5 / 100
data['datetime'] = pd.to_datetime(data['datetime'])
# Process the data day by day
data['date'] = data['datetime'].dt.date
unique_dates = data['date'].unique()

for date in unique_dates:
    day_data = data[data['date'] == date]

    # Get the first row of the day for gap opening analysis
    first_row = day_data.iloc[0]
    previous_day = data[data['date'] < date].iloc[-1] if len(data[data['date'] < date]) > 0 else None

    if previous_day is None:
        continue

    vah = previous_day['vah']
    val = previous_day['val']
    poc = previous_day['poc']

    # Flags for trends and moves to POC
    bullish_flag = False
    bearish_flag = False
    poc_flag = False

    # Track the highest and lowest prices of the day
    day_high = day_data['high'].max()
    day_low = day_data['low'].min()

    # Identify gap openings
    if first_row['open'] > vah:
        # Check for bullish trend
        if day_high >= vah * (1 + threshold):
            bullish_flag = True
    elif first_row['open'] < val:
        # Check for bearish trend
        if day_low <= val * (1 - threshold):
            bearish_flag = True

    # Check for move to POC
    if day_low <= poc <= day_high:
        poc_flag = True

    # Update the counters based on flags
    if bullish_flag:
        bullish_trends += 1
    if bearish_flag:
        bearish_trends += 1
    if poc_flag:
        moves_to_poc += 1

# Output the results
print(f"Bullish Trends: {bullish_trends}")
print(f"Bearish Trends: {bearish_trends}")
print(f"Moves to POC: {moves_to_poc}")

Bullish Trends: 15
Bearish Trends: 119
Moves to POC: 726


In [ ]:
data=data_
data['datetime'] = pd.to_datetime(data['datetime'])

# Initialize counters for different thresholds and total days
bullish_moves = {'1%': 0, '1.5%': 0, '2%': 0, '>2.5%': 0}
bearish_moves = {'1%': 0, '1.5%': 0, '2%': 0, '>2.5%': 0}
total_days = 0
profitable_days = 0

# Define the thresholds
thresholds = [1 / 100, 1.5 / 100, 2 / 100, 2.5 / 100]

# Process the data day by day
data['date'] = data['datetime'].dt.date
unique_dates = data['date'].unique()

for date in unique_dates:
    day_data = data[data['date'] == date]

    # Get the first row of the day for gap opening analysis
    first_row = day_data.iloc[0]
    previous_day_data = data[data['date'] < date]
    previous_day = previous_day_data.iloc[-1] if not previous_day_data.empty else None

    if previous_day is None:
        continue

    vah = previous_day['vah']
    val = previous_day['val']
    poc = previous_day['poc']

    # Flags for moves to POC
    poc_flag = False

    # Identify gap openings and check for trends
    if first_row['open'] > vah:
        # Check for bullish movements
        for _, row in day_data.iterrows():
            for threshold in thresholds:
                if row['close'] >= vah * (1 + threshold):
                    if threshold == thresholds[0]:
                        bullish_moves['1%'] += 1
                    elif threshold == thresholds[1]:
                        bullish_moves['1.5%'] += 1
                    elif threshold == thresholds[2]:
                        bullish_moves['2%'] += 1
                    else:
                        bullish_moves['>2.5%'] += 1
                    profitable_days += 1
                    break  # Stop after counting the highest threshold achieved
    elif first_row['open'] < val:
        # Check for bearish movements
        for _, row in day_data.iterrows():
            for threshold in thresholds:
                if row['close'] <= val * (1 - threshold):
                    if threshold == thresholds[0]:
                        bearish_moves['1%'] += 1
                    elif threshold == thresholds[1]:
                        bearish_moves['1.5%'] += 1
                    elif threshold == thresholds[2]:
                        bearish_moves['2%'] += 1
                    else:
                        bearish_moves['>2.5%'] += 1
                    profitable_days += 1
                    break  # Stop after counting the highest threshold achieved

    # Check for move to POC
    for _, row in day_data.iterrows():
        if not poc_flag and row['low'] <= poc <= row['high']:
            poc_flag = True
            break

    if poc_flag:
        total_days += 1

# Calculate total days
total_days = len(unique_dates)

# Calculate percentage of profitable days
percentage_profitable = (profitable_days / total_days) * 100 if total_days > 0 else 0

# Output the results
print("Bullish Moves:")
for threshold, count in bullish_moves.items():
    print(f"  {threshold}: {count}")

print("Bearish Moves:")
for threshold, count in bearish_moves.items():
    print(f"  {threshold}: {count}")

print(f"Total Days: {total_days}")
print(f"Percentage of Profitable Days: {percentage_profitable:.2f}%")

Bullish Moves:
  1%: 676
  1.5%: 0
  2%: 0
  >2.5%: 0
Bearish Moves:
  1%: 7548
  1.5%: 0
  2%: 0
  >2.5%: 0
Total Days: 1130
Percentage of Profitable Days: 727.79%


# 3-Day 5-Day Tight profile breakout

In [ ]:
df = pd.DataFrame(data_)
df['datetime'] = pd.to_datetime(df['datetime'])
columns_to_check = ['3_day_vah', '5_day_vah', '3_day_val', '5_day_val', '3_day_poc', '5_day_poc']
nan_dates = df[df[columns_to_check].isna().any(axis=1)]['datetime']
unique_nan_dates = pd.to_datetime(nan_dates.dt.date).drop_duplicates().tolist()
for date in unique_nan_dates:
    print(date.strftime('%Y-%m-%d'))

2018-01-01
2018-01-02
2018-01-03
2018-01-04
2019-01-01
2019-01-02
2019-01-03
2019-01-04
2020-01-01
2020-01-02
2020-01-03
2020-01-06
2021-01-01
2021-01-04
2021-01-05
2021-01-06
2022-01-03
2022-01-04
2022-01-05
2022-01-06


In [ ]:
import pandas as pd
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
def add_breakout_flags(data, width_threshold=0.01, move_threshold=0.01):
    data['breakout_flag'] = 0
    breakout_up_count = 0
    breakout_down_count = 0

    for date, group in data.groupby(data['datetime'].dt.date):
        prev_vah = group.iloc[-1]['3_day_vah']
        prev_val = group.iloc[-1]['3_day_val']
        width = prev_vah - prev_val

        if width / prev_vah < width_threshold:
            close_prices = group['close']
            breakout_up = close_prices > prev_vah * (1 + move_threshold)
            breakout_down = close_prices < prev_val * (1 - move_threshold)

            if breakout_up.any():
                breakout_up_count += 1

            if breakout_down.any():
                breakout_down_count += 1

    return breakout_up_count, breakout_down_count

breakout_up_count, breakout_down_count = add_breakout_flags(data, width_threshold=0.01, move_threshold=0.01)
total_tight_days = breakout_up_count + breakout_down_count

print(f'Total number of days where the previous two days had a narrow width: {total_tight_days}')
print(f'Number of breakout days (breakup): {breakout_up_count}')
print(f'Number of breakout days (breakdown): {breakout_down_count}')


Total number of days where the previous two days had a narrow width: 128
Number of breakout days (breakup): 62
Number of breakout days (breakdown): 66


In [ ]:
import pandas as pd

data = data_
data['datetime'] = pd.to_datetime(data['datetime'])

def add_breakout_flags(data, width_thresholds=[0.01], move_threshold=0.01):
    breakout_counts = {}

    for width_threshold in width_thresholds:
        breakout_up_count = 0
        breakout_down_count = 0

        for date, group in data.groupby(data['datetime'].dt.date):
            prev_vah = group.iloc[-1]['3_day_vah']
            prev_val = group.iloc[-1]['3_day_val']
            width = prev_vah - prev_val

            if width / prev_vah < width_threshold:
                close_prices = group['close']
                breakout_up = close_prices > prev_vah * (1 + move_threshold)
                breakout_down = close_prices < prev_val * (1 - move_threshold)

                if breakout_up.any():
                    breakout_up_count += 1

                if breakout_down.any():
                    breakout_down_count += 1

        breakout_counts[width_threshold] = (breakout_up_count, breakout_down_count)

    return breakout_counts

width_thresholds = [0.01, 0.015, 0.02, 0.025]
breakout_counts = add_breakout_flags(data, width_thresholds=width_thresholds, move_threshold=0.01)

for width_threshold, counts in breakout_counts.items():
    total_tight_days = sum(counts)
    breakout_up_count, breakout_down_count = counts
    print(f'For width threshold {width_threshold}:')
    print(f'Total number of days where the previous two days had a narrow width: {total_tight_days}')
    print(f'Number of breakout days (breakup): {breakout_up_count}')
    print(f'Number of breakout days (breakdown): {breakout_down_count}')
    print("____________________________________________________________________________________")

For width threshold 0.01:
Total number of days where the previous two days had a narrow width: 128
Number of breakout days (breakup): 62
Number of breakout days (breakdown): 66
____________________________________________________________________________________
For width threshold 0.015:
Total number of days where the previous two days had a narrow width: 274
Number of breakout days (breakup): 104
Number of breakout days (breakdown): 170
____________________________________________________________________________________
For width threshold 0.02:
Total number of days where the previous two days had a narrow width: 349
Number of breakout days (breakup): 123
Number of breakout days (breakdown): 226
____________________________________________________________________________________
For width threshold 0.025:
Total number of days where the previous two days had a narrow width: 382
Number of breakout days (breakup): 133
Number of breakout days (breakdown): 249
_____________________________

from which point it has reverted and if there was any high pe or ce over there???

In [ ]:
import pandas as pd

# Load the data
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])

# Function to add breakout flags based on 3-day VAH and VAL
def add_breakout_flags(data, width_thresholds=[0.01], move_threshold=0.01):
    breakout_counts = {}

    for width_threshold in width_thresholds:
        breakout_up_count = 0
        breakout_down_count = 0
        breakout_crossed_counts = {
            '< 1%': 0,
            '1-1.25%': 0,
            '1.25-1.5%': 0,
            '1.5-2%': 0,
            '> 2%': 0
        }

        for date, group in data.groupby(data['datetime'].dt.date):
            prev_vah = group.iloc[-1]['3_day_vah']
            prev_val = group.iloc[-1]['3_day_val']
            width = prev_vah - prev_val

            if width / prev_vah < width_threshold:
                close_prices = group['close']
                breakout_up = close_prices > prev_vah * (1 + move_threshold)
                breakout_down = close_prices < prev_val * (1 - move_threshold)

                if breakout_up.any():
                    breakout_up_count += 1
                    breakout_price = close_prices[breakout_up.idxmax()]  # Price at breakout
                    percentage_change = (close_prices.iloc[-1] - breakout_price) / breakout_price * 100
                    if percentage_change < 1:
                        breakout_crossed_counts['< 1%'] += 1
                    elif 1 <= percentage_change < 1.25:
                        breakout_crossed_counts['1-1.25%'] += 1
                    elif 1.25 <= percentage_change < 1.5:
                        breakout_crossed_counts['1.25-1.5%'] += 1
                    elif 1.5 <= percentage_change < 2:
                        breakout_crossed_counts['1.5-2%'] += 1
                    else:
                        breakout_crossed_counts['> 2%'] += 1

                if breakout_down.any():
                    breakout_down_count += 1
                    breakout_price = close_prices[breakout_down.idxmax()]  # Price at breakout
                    percentage_change = (close_prices.iloc[-1] - breakout_price) / breakout_price * 100
                    if percentage_change < -1:
                        breakout_crossed_counts['< 1%'] += 1
                    elif -1.25 <= percentage_change < -1:
                        breakout_crossed_counts['1-1.25%'] += 1
                    elif -1.5 <= percentage_change < -1.25:
                        breakout_crossed_counts['1.25-1.5%'] += 1
                    elif -2 <= percentage_change < -1.5:
                        breakout_crossed_counts['1.5-2%'] += 1
                    else:
                        breakout_crossed_counts['> 2%'] += 1

        breakout_counts[width_threshold] = (breakout_up_count, breakout_down_count, breakout_crossed_counts)

    return breakout_counts

# Add breakout flags
width_thresholds = [0.01, 0.015, 0.02, 0.025]
breakout_counts = add_breakout_flags(data, width_thresholds=width_thresholds, move_threshold=0.01)

# Output the results
for width_threshold, counts in breakout_counts.items():
    breakout_up_count, breakout_down_count, breakout_crossed_counts = counts
    print(f'For width threshold {width_threshold}:')
    print(f'Total number of days where the previous two days had a narrow width: {breakout_up_count + breakout_down_count}')
    print(f'Number of breakout days (breakup): {breakout_up_count}')
    print(f'Number of breakout days (breakdown): {breakout_down_count}')
    print(f'Breakout crossed counts (breakup):')
    for threshold, count in breakout_crossed_counts.items():
        print(f'{threshold}: {count}')


For width threshold 0.01:
Total number of days where the previous two days had a narrow width: 128
Number of breakout days (breakup): 62
Number of breakout days (breakdown): 66
Breakout crossed counts (breakup):
< 1%: 63
1-1.25%: 0
1.25-1.5%: 0
1.5-2%: 0
> 2%: 65
For width threshold 0.015:
Total number of days where the previous two days had a narrow width: 274
Number of breakout days (breakup): 104
Number of breakout days (breakdown): 170
Breakout crossed counts (breakup):
< 1%: 110
1-1.25%: 1
1.25-1.5%: 2
1.5-2%: 2
> 2%: 159
For width threshold 0.02:
Total number of days where the previous two days had a narrow width: 349
Number of breakout days (breakup): 123
Number of breakout days (breakdown): 226
Breakout crossed counts (breakup):
< 1%: 134
1-1.25%: 3
1.25-1.5%: 2
1.5-2%: 3
> 2%: 207
For width threshold 0.025:
Total number of days where the previous two days had a narrow width: 382
Number of breakout days (breakup): 133
Number of breakout days (breakdown): 249
Breakout crossed co

In [ ]:
import pandas as pd
import numpy as np

# Load the data
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])

# Function to add breakout flags based on 3-day VAH and VAL
def add_breakout_flags(data, width_thresholds=[0.01], move_threshold=0.01):
    breakout_counts = {}

    for width_threshold in width_thresholds:
        breakout_up_count = 0
        breakout_down_count = 0
        breakout_crossed_counts = {
            '< 1%': 0,
            '1-1.5%': 0,
            '1.5-2%': 0,
            '2-2.5%': 0,
            '2.5-3%': 0,
            '> 3%': 0
        }

        for date, group in data.groupby(data['datetime'].dt.date):
            prev_vah = group.iloc[-1]['3_day_vah']
            prev_val = group.iloc[-1]['3_day_val']
            width = prev_vah - prev_val

            if width / prev_vah < width_threshold:
                close_prices = group['close']
                breakout_up = close_prices > prev_vah * (1 + move_threshold)
                breakout_down = close_prices < prev_val * (1 - move_threshold)

                if breakout_up.any():
                    breakout_up_count += 1
                    breakout_price = close_prices[breakout_up.idxmax()]  # Price at breakout
                    percentage_change = (close_prices.iloc[-1] - breakout_price) / breakout_price * 100
                    if percentage_change < 1:
                        breakout_crossed_counts['< 1%'] += 1
                    elif 1 <= percentage_change <= 1.5:
                        breakout_crossed_counts['1-1.5%'] += 1
                    elif 1.5 <= percentage_change <= 2:
                        breakout_crossed_counts['1.5-2%'] += 1
                    elif 2 <= percentage_change <= 2.5:
                        breakout_crossed_counts['2-2.5%'] += 1
                    elif 2.5 <= percentage_change <= 3:
                        breakout_crossed_counts['2.5-3%'] += 1
                    else:
                        breakout_crossed_counts['> 3%'] += 1

                if breakout_down.any():
                    breakout_down_count += 1
                    breakout_price = close_prices[breakout_down.idxmax()]  # Price at breakout
                    percentage_change = (close_prices.iloc[-1] - breakout_price) / breakout_price * 100
                    if percentage_change > -1:
                        breakout_crossed_counts['< 1%'] += 1
                    elif -1.5 <= percentage_change < -1:
                        breakout_crossed_counts['1-1.5%'] += 1
                    elif -2 <= percentage_change < -1.5:
                        breakout_crossed_counts['1.5-2%'] += 1
                    elif -2.5 <= percentage_change < -2:
                        breakout_crossed_counts['2-2.5%'] += 1
                    elif -3 <= percentage_change < -2.5:
                        breakout_crossed_counts['2.5-3%'] += 1
                    else:
                        breakout_crossed_counts['> 3%'] += 1

        breakout_counts[width_threshold] = (breakout_up_count, breakout_down_count, breakout_crossed_counts)

    return breakout_counts

# Add breakout flags
width_thresholds = [0.01, 0.015, 0.02, 0.025]
breakout_counts = add_breakout_flags(data, width_thresholds=width_thresholds, move_threshold=0.01)

# Output the results
for width_threshold, counts in breakout_counts.items():
    breakout_up_count, breakout_down_count, breakout_crossed_counts = counts
    print(f'For width threshold {width_threshold}:')
    print(f'Total number of days where the previous two days had a narrow width: {breakout_up_count + breakout_down_count}')
    print(f'Number of breakout days (breakup): {breakout_up_count}')
    print(f'Number of breakout days (breakdown): {breakout_down_count}')
    print(f'Breakout crossed counts (breakup):')
    for threshold, count in breakout_crossed_counts.items():
        print(f'{threshold}: {count}')


For width threshold 0.01:
Total number of days where the previous two days had a narrow width: 128
Number of breakout days (breakup): 62
Number of breakout days (breakdown): 66
Breakout crossed counts (breakup):
< 1%: 127
1-1.5%: 1
1.5-2%: 0
2-2.5%: 0
2.5-3%: 0
> 3%: 0
For width threshold 0.015:
Total number of days where the previous two days had a narrow width: 274
Number of breakout days (breakup): 104
Number of breakout days (breakdown): 170
Breakout crossed counts (breakup):
< 1%: 256
1-1.5%: 10
1.5-2%: 6
2-2.5%: 2
2.5-3%: 0
> 3%: 0
For width threshold 0.02:
Total number of days where the previous two days had a narrow width: 349
Number of breakout days (breakup): 123
Number of breakout days (breakdown): 226
Breakout crossed counts (breakup):
< 1%: 318
1-1.5%: 17
1.5-2%: 9
2-2.5%: 4
2.5-3%: 0
> 3%: 1
For width threshold 0.025:
Total number of days where the previous two days had a narrow width: 382
Number of breakout days (breakup): 133
Number of breakout days (breakdown): 249
Bre